In [1]:
import pyodbc
import pandas as pd
import platform
import datetime

import getpass

USERNAME = getpass.getpass(prompt="Enter your QID here: ")
PASSWORD = getpass.getpass(prompt="Enter your password here: ")
DSN = 'Teradata'
connection = pyodbc.connect('DSN=' + DSN + '; UID=' + USERNAME + '; PWD='+ PASSWORD)

Enter your QID here:  ·······
Enter your password here:  ·············


# Config

In [2]:
root_data = "/apps/pproapp0/proews/current/jupyter/ey_ews_data/"
processed_path = root_data+"processed_data/"
raw_data_dir = root_data + "ey_raw_data/"
interm_data_dir = root_data+"intermediate/"

In [3]:
# Set character encoding/decoding.
# This ensures our server can communicate with the Teradata server properly.
# If you don't set these, the query will fail.
# The reason is because Teradata represents strings differently than our server. 
connection.setdecoding(pyodbc.SQL_CHAR, encoding="utf_16_le")
connection.setdecoding(pyodbc.SQL_WCHAR, encoding="utf_16_le")
connection.setdecoding(pyodbc.SQL_WMETADATA, encoding="utf_16_le")
connection.setencoding(encoding="utf_16_le")

#### The next cell are similarly structured, they contain the core SQL query for each of the following tables:
 - Client_General
 - Mort_hist
 - Loan_Hist
 - Lend_General
 - Financials
 - Fac_general
 - BDA General
 - Prod Char
 - Bhvr Char
 - BRR Extract
 - BDA TXN NO COBC
 - Loan General
 - Agg Loan TXN Acct Level

# Client_General

In [4]:
def pull_client_general(start_snap_dt, end_snap_dt):    
    qry = f"""\
        SELECT 
        /* This datapull uses RB_CLNT as the base population for pulling data */
        /* month end date */
               C.SNAP_DT as mth_end_dt, 
        /* client number */
               C.clnt_no , 
        /* client group, used to define commercial and small business clients */
        case
                    when (c.BSC in (190,192,200,202,250,490,590)
                            or (c.BSC in (310,500,502,504,505,510) and 
                                (c.CURR_CMI_NO not in (100,101,102,103,
                                                        200,201,202,203,
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503) or c.CURR_CMI_NO is NULL))
                         ) then 'COMM'
            
                    when (c.BSC in (550,554,555)
                            and (c.CURR_CMI_NO not in (100,101,102,103,
                                                        200,201,202,203,
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503) or c.CURR_CMI_NO is NULL)
                         ) then 'AG'
                    when c.BSC in (330) then 'PUB'
                    when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                            and c.CURR_CMI_NO in (100,101,102,103,
                                                    200,201,202,203,
                                                    300,301,302,303,
                                                    401,402,403,
                                                    501,502,503)
                         )then 'SB'
                    else 'OOO'
                end as Clnt_Grp,
        /* response transit center*/
               C.resp_unt AS resp_transit,
        /*BSC code */
               C.BSC, 
        /*Current client management index*/
               C.CURR_CMI_NO AS cmi,
        /*country*/
               CASE
                 WHEN c.snap_dt LE '2007-09-30' THEN ''
                 WHEN AA.prov_cd IN ('AB','BC','MB','NB','NL','NS','NT','NU','ON','PE','QC','SK','YT') THEN 'Canada'
                 ELSE 'Others'
               END AS country, 
        /*STATE/COUNTRY NAME*/
               AA.st_cntry_nm, 
        /*SUB COUNTRY CODE OF RESIDENCE (SUB_CNTRY_CD)*/
               AA.prov_cd, 
        /*Critieria Paper BORROWER RISK RATING (BRR) ( RSK_RTG )*/
               CH.RSK_RTG as BRR, 
        /* Numerical version of BRR */
            CASE
                        WHEN CH.RSK_RTG = '1+' THEN 1
                        WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD = 'H' THEN 2
                        WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD = 'M'  THEN 3
                        WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD IS NULL THEN 3
                        WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD = 'L' THEN 4
                        WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD = 'H' THEN 5
                        WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD = 'M' THEN 6
                        WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD IS NULL THEN 6
                        WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD = 'L' THEN 7
                        WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD = 'H' THEN 8
                        WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD = 'M' THEN 9
                        WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD IS NULL THEN 9
                        WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD = 'L' THEN 10
                        WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD = 'H' THEN 11
                        WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD = 'M' THEN 12
                        WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD IS NULL THEN 12
                        WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD = 'L' THEN 13
                        WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD = 'H' THEN 14
                        WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD = 'M' THEN 15
                        WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD IS NULL THEN 15
                        WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD = 'L' THEN 16
                        WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD = 'H' THEN 17
                        WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD = 'M' THEN 18
                        WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD IS NULL THEN 18
                        WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD = 'L' THEN 19
                        WHEN CH.RSK_RTG = '4' THEN 20
                        WHEN CH.RSK_RTG = '5' THEN 21
                        WHEN CH.RSK_RTG = '6' THEN 22
                      END as num_brr,
        /*Special Loan Indicator*/
               case
                 when BB.clnt_no is not null then 'Y' /* if client exists in the BB table, then it is SLG client */
                 else 'N'
                 end as slg_ind,
        /* single name ID */
                 CASE
                       WHEN SN.SN_ID IS NULL THEN C.CLNT_NO
                       ELSE SN.SN_ID
                     END AS SN_ID,
        /* sic number */
               CH.sic_cd1 AS sic,
        /* sic description */
               S.sic_desc_en,
        /*CUSTOMER RISK INDEX GENERAL (CRI_GNRL)*/
               D.cri_gnrl,
        /*DATE BUSINESS ESTABLISHED*/
               B.dt_bus_estb,
        /*VERTICAL INDUSTRY CLASSIFICATION CODE*/
               V.vic,  
        /*VERTICAL INDUSTRY CODE IDENTIFIER*/
               V.vic_id,
        /*VERTICAL INDUSTRY CLASSIFICATION CODE DESCRIPTION*/
               V.vic_desc_en, 
        /*SUB-VERTICAL INDUSTRY CLASSIFICATION CODE*/
               V.sub_vic, 
        /*SUB-VERTICAL INDUSTRY CODE IDENTIFIER*/
               V.sub_vic_id,
        /*SUB-VERTICAL INDUSTRY CLASSIFICATION CODE DESCRIPT*/
               V.sub_vic_desc_en, 
        /* Personal/business customer type flag from Probe */
                D.pers_bus_flg_typ,
        /* Date client signed up with RBC*/
                ch.dt_opened,
        /* Number of years being RBC client */
                (ch.mth_end_dt - ch.DT_OPENED)/365  AS yrs_being_client,
        /*BRR REVIEW DATE ( BRR_RVW_DT )*/
               C.brr_rvw_dt AS brr_review_dt, 
        /* Client status (Active/Inactive) with 1 month lag*/
               C.clnt_sts as status,
        /*Client Total Authorization*/
               CASE WHEN BPC.clnt_auth is not null then BPC.clnt_auth
                    ELSE 0
                end as clnt_auth, 
        /*Client Total Outstanding*/
               CASE WHEN BPC.clnt_os is not null then BPC.clnt_os
                    ELSE 0
                end as clnt_os,
        /*Borrower/Non-Borrower Indicator*/
               CASE
                 WHEN (BPC.clnt_os NE 0 and bpc.clnt_os is not null) 
                        OR (BPC.clnt_auth NE 0 and bpc.clnt_auth is not null) 
                        OR OD.od_ind=1 THEN 1
                 ELSE 0
               END AS borrower_ind,
        /*Client total single name exposure*/
                CASE WHEN BPC.tot_sn_exp is not null then BPC.tot_sn_exp
                    ELSE 0
                end as tot_sn_exp,
        /*Maximum Historical Total Single Name Exposure*/
               case when T.max_tsne is not null then t.max_tsne
               else 0 
               end as max_tsne
        
        
        FROM DDWV01.rb_clnt AS C
        
                /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
             INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                  ON  r.SNAP_DT  = c.snap_dt
                      AND r.ORG_UNT_ID = c.resp_unt
                      AND r.PARNT_HIER_TYP_ID = 80000 
                      AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)
        
            /* Identify clients with SLG manager, logic obtained from Business Portfolio Client table source code */
             LEFT JOIN
             (
                 select distinct h.clnt_no, h.assign_typ, h.acct_mgr_cd,h.mth_end_dt
                 from dg6v19.clnt_am_assignment_hist as H
                 inner join ddwv01.org_unt_to_slg_mgr_reltn as S
                 on H.acct_mgr_cd = S.slg_tr_no
                 and H.mth_end_dt > '{start_snap_dt}' AND H.mth_end_dt < '{end_snap_dt}'
                 and H.assign_typ = 'S'
                  ) as BB
                    ON C.clnt_no = BB.clnt_no
                    and C.snap_dt = BB.mth_end_dt
        
            /* Used to get client SN ID, logic obtained from Business Portfolio Client table source code  */
            LEFT JOIN DG6V19.CLNT_SN_HIST as SN
                         ON C.CLNT_NO = SN.CLNT_NO
                        AND C.snap_DT = SN.MTH_END_DT
        
            /* use this table to get the link to address, VIC tables, SIC tables */
             LEFT JOIN DG6V19.clnt_hist AS CH
               ON CH.clnt_no = C.clnt_no
                  AND CH.mth_end_dt = c.snap_dt
        
            /* SIC info */
             LEFT JOIN DDWV01.sic AS S
               ON S.sic_cd = CH.sic_cd1
                  AND S.snap_dt = CH.mth_end_dt
        
            /* Business established date */
             LEFT JOIN DG6V19.non_pers_clnt_hist B
               ON B.clnt_no = C.clnt_no
                  AND B.mth_end_dt = c.snap_dt
        
            /* Address */
             LEFT JOIN DG6V19.addr_hist AS AA
               ON AA.addr_id = CH.addr_id
                  AND AA.mth_end_dt = CH.mth_end_dt
                  AND AA.addr_src = 'C'
            
            /* VIC */	
             LEFT JOIN DG6V19.sub_vic_hist AS V
               ON V.sub_vic_id = CH.sub_vic_id
                  AND V.mth_end_dt = CH.mth_end_dt  
        
            /* Probe table */
             LEFT JOIN DDWV11.clnt_bhvr_charstc AS D
               ON D.clnt_no = C.clnt_no
                  AND D.capture_dt = c.snap_dt 
                  AND D.pers_bus_flg_typ = 'B'
        
            /* table to identify if client has overdraft facility */
            left join ( select clnt_no, mth_end_dt, max(od_ind) as od_ind /* aggregate to client level */
                        from 
                        (select lend.clnt_no, /* arrangement level table */
                              lend.mth_end_dt,
                              lend.ar_id,
                              CASE WHEN FAC_TABLE.FAC_TYPE1 in ('OD', 'OV', 'OM') THEN 1
                                ELSE 0
                              END AS od_ind
                            /* Arrangement ownership history table to link accounts to clients*/
                              from DG6V19.ARNGMNT_OWN_HIST as lend 
                              /* Facility table to identify overdraft facilities */
                              inner join DG6V19.BUS_COMM_RPT_FAC as FAC_TABLE
                              on FAC_TABLE.prod_no = lend.ar_id
                              and FAC_TABLE.SNAP_DT = lend.MTH_END_DT
        
                              where FAC_TABLE.SNAP_DT > '{start_snap_dt}' AND FAC_TABLE.SNAP_DT < '{end_snap_dt}'
                              ) as OD_fac 
                            group by 1, 2) as OD
                  ON C.CLNT_NO = OD.CLNT_NO 
                  AND c.snap_dt = OD.MTH_END_DT
        
             /* Calculate max TSNE */
             LEFT JOIN (SELECT t1.MTH_END_DT,  
                                t1.CLNT_NO, 
                                Max(t2.TOT_SN_EXP) AS Max_TSNE 
                         FROM 
                                DG6V19.BUS_PRTFOL_CLNT AS t1,
                                DG6V19.BUS_PRTFOL_CLNT AS t2
                         WHERE
                                t1.CLNT_NO = t2.CLNT_NO AND 
                                t1.MTH_END_DT >= t2.MTH_END_DT 
                         GROUP BY 1,2 )AS T
            
               ON C.CLNT_NO = T.CLNT_NO 
                  AND c.snap_dt = T.MTH_END_DT
            
            /* business portfolio client table with client authorization, outstanding, max TSNE info */
             LEFT JOIN DG6V19.BUS_PRTFOL_CLNT AS BPC
               ON BPC.clnt_no = C.clnt_no
                  AND BPC.mth_end_dt = c.snap_dt
        
        WHERE CLNT_GRP <> 'OOO' /* filter out non-COMM/SB clients*/
        And c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'  /* get info on snap date */
        and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
        and c.clnt_typ=2 /* keep only non-personal clients */
        and BPC.tot_sn_exp > 1000000
        """
    return pd.read_sql(qry, connection)

In [5]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_client_general(start_snap_dt, end_snap_dt)
df1.shape

(448856, 33)

In [6]:
df1.head()

,mth_end_dt,CLNT_NO,Clnt_Grp,resp_transit,BSC,cmi,country,ST_CNTRY_NM,PROV_CD,BRR,...,PERS_BUS_FLG_TYP,DT_OPENED,yrs_being_client,brr_review_dt,status,clnt_auth,clnt_os,borrower_ind,tot_sn_exp,max_tsne
0,2015-01-31,528885403.0,COMM,5471,505,400,Canada,None,ON,2,...,B,2010-04-01,4,2015-06-30,A,1501889.29,1501889.29,1,1501889.29,1950000.00
1,2015-05-31,846087534.0,COMM,4482,505,400,Canada,None,ON,3+,...,B,2001-12-01,13,2013-09-30,A,0.00,0.00,0,2050000.00,3061710.68
2,2015-01-31,542884630.0,COMM,4756,505,304,Canada,None,ON,3+,...,B,2013-03-01,1,2015-01-31,A,962763.00,960185.15,1,2017770.00,3378983.50
3,2015-04-30,780242707.0,PUB,13908,330,400,Canada,None,ON,2+,...,None,1990-07-01,24,2015-12-31,A,13025001.00,0.00,1,13070001.00,13250000.00
4,2015-01-31,313757106.0,COMM,8966,200,400,Canada,None,ON,2-,...,B,2014-11-01,0,2015-05-31,A,1435000.00,1435000.00,1,45115296.51,45116476.38


In [7]:
df1.drop_duplicates(subset=['mth_end_dt', 'CLNT_NO'], keep='first', inplace=True)

In [8]:
df1.shape

(448856, 33)

In [9]:
df1.to_csv(raw_data_dir+'clnt_general_y1.csv', index=False)

In [10]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_client_general(start_snap_dt, end_snap_dt)
df2.shape

(471303, 33)

In [11]:
df2.drop_duplicates(subset=['mth_end_dt', 'CLNT_NO'], keep='first', inplace=True)
df2.shape

(471303, 33)

In [12]:
df2.to_csv(raw_data_dir+'clnt_general_y2.csv', index=False)

In [13]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_client_general(start_snap_dt, end_snap_dt)
df3.shape

(499314, 33)

In [14]:
df3.drop_duplicates(subset=['mth_end_dt', 'CLNT_NO'], keep='first', inplace=True)
df3.shape

(499314, 33)

In [15]:
df3.to_csv(raw_data_dir+'clnt_general_y3.csv', index=False)

In [16]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_client_general(start_snap_dt, end_snap_dt)
df4.shape

(809246, 33)

In [17]:
df4.drop_duplicates(subset=['mth_end_dt', 'CLNT_NO'], keep='first', inplace=True)
df4.shape

(809246, 33)

In [18]:
df4.to_csv(raw_data_dir+'clnt_general_y4.csv', index=False)

# Mort_hist

In [19]:
def pull_mort_hist(start_snap_dt, end_snap_dt):    
    qry = f"""\
            select 
            /* month end date */
                   C.SNAP_DT as mth_end_dt, 
            /* client number */
                   C.clnt_no , 
            /* client group, used to define commercial and small business clients */
            case
                        when (c.BSC in (190,192,200,202,250,490,590)
                                or (c.BSC in (310,500,502,504,505,510) and 
                                    (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203,
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL))
                             ) then 'COMM'
                
                        when (c.BSC in (550,554,555)
                                and (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL)
                             ) then 'AG'
                        when c.BSC in (330) then 'PUB'
                        when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                and c.CURR_CMI_NO in (100,101,102,103,
                                                        200,201,202,203,
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503)
                             )then 'SB'
                        else 'OOO'
                    end as Clnt_Grp,
                   L.ar_id ,   /*-- arrangement id*/
                   'A' || trim(L.ar_id) as t_ar_id, /* added by QH on 2018-11-07 */
                   L.mif_srvc_id, /*--MIF service identifier,*/
                   MT.LST_FNCL_TXN_DT,
                   case
                     when extract (YEAR From MT.LST_FNCL_TXN_DT ) = Extract (YEAR From c.snap_dt)
                              and  extract (MONTH From MT.LST_FNCL_TXN_DT) = Extract (MONTH From c.snap_dt) then 1
                     else 0
                    end as lst_fncl_txn_dt_ind,
                    MT.WRITE_OFF_DATE,
                    case 
                      when extract (YEAR From MT.WRITE_OFF_DATE ) = Extract (YEAR From c.snap_dt)
                              and  extract (MONTH From MT.WRITE_OFF_DATE) = Extract (MONTH From c.snap_dt) then 1
                     else 0
                    end as wo_ind
                    
            
            FROM DDWV01.rb_clnt AS C
            /*Neo Added Logic */
                     INNER JOIN DG6V19.BUS_PRTFOL_CLNT AS BCP
                        ON  C.SNAP_DT  = BCP.MTH_END_DT
                        AND c.clnt_no = BCP.CLNT_NO 
                        
                    /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                 INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                      ON  r.SNAP_DT  = c.snap_dt
                          AND r.ORG_UNT_ID = c.resp_unt
                          AND r.PARNT_HIER_TYP_ID = 80000 
                          AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)
            
                  INNER JOIN DG6V19.bus_prtfol_lend as L
                     ON L.clnt_no = C.clnt_no
                         and L.mth_end_dt = C.snap_dt
                    /*     --and L.mth_end_dt LE '2018-05-31'
                         --and L.mth_end_dt >= '2016-05-31'*/
            
                  INNER JOIN DDWV01.MORTG_FINANCL_HIST as MT
                    on MT.snap_dt = C.snap_dt
                       and MT.DW_SRVC_ID = L.MIF_SRVC_ID
                       and L.MIF_SRVC_ID = 21
                       and MT.AR_ID = L.AR_ID
                          
            
            where clnt_grp NOT IN ('OOO', 'SB')
                  AND TOT_SN_EXP > 1000000
                  and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                  and c.clnt_typ=2 /* keep only non-personal clients */
            and C.snap_dt > '{start_snap_dt}' AND C.snap_dt < '{end_snap_dt}'
    """
    return pd.read_sql(qry, connection)

In [20]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_mort_hist(start_snap_dt, end_snap_dt)
df1.shape

(184833, 10)

In [21]:
df1.to_csv(raw_data_dir+'mort_hist_y1.csv', index=False)

In [22]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_mort_hist(start_snap_dt, end_snap_dt)
df2.shape

(192398, 10)

In [23]:
df2.to_csv(raw_data_dir+'mort_hist_y2.csv', index=False)

In [24]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_mort_hist(start_snap_dt, end_snap_dt)
df3.shape

(204829, 10)

In [25]:
df3.to_csv(raw_data_dir+'mort_hist_y3.csv', index=False)

In [26]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_mort_hist(start_snap_dt, end_snap_dt)
df4.shape

(324810, 10)

In [27]:
df4.to_csv(raw_data_dir+'mort_hist_y4.csv', index=False)

# Loan_Hist

In [28]:
def pull_loan_hist(start_snap_dt, end_snap_dt):    
    qry = f"""\
                    select 
                    /* month end date */
                           C.SNAP_DT as mth_end_dt, 
                    /* client number */
                           C.clnt_no , 
                    /* client group, used to define commercial and small business clients */
                    case
                                when (c.BSC in (190,192,200,202,250,490,590)
                                        or (c.BSC in (310,500,502,504,505,510) and 
                                            (c.CURR_CMI_NO not in (100,101,102,103,
                                                                    200,201,202,203, 
                                                                    300,301,302,303,
                                                                    401,402,403,
                                                                    501,502,503) or c.CURR_CMI_NO is NULL))
                                     ) then 'COMM'

                                when (c.BSC in (550,554,555)
                                        and (c.CURR_CMI_NO not in (100,101,102,103,
                                                                    200,201,202,203, 
                                                                    300,301,302,303,
                                                                    401,402,403,
                                                                    501,502,503) or c.CURR_CMI_NO is NULL)
                                     ) then 'AG'
                                when c.BSC in (330) then 'PUB'
                                when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                        and c.CURR_CMI_NO in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503)
                                     )then 'SB'
                                else 'OOO'
                            end as Clnt_Grp,
                          A.ar_id, /*-- arrangement id*/
                          'A' || trim(A.ar_id) as t_ar_id, /* Added by QH 2018-11-08*/
                          A.acct_sts_eff_dt, /*--account status effective date*/
                          A.activatn_dt, /*-- activation date*/
                          A.amort_end_dt, /*-- amortization end date - current*/
                          A.amort_end_dt_orgnl, /*-- amortization end date - original*/
                         A.clctn_ind,  /*-- collection indicator*/
                         A.clctn_reas_cd, /*-- collection reason code*/
                         A.tr_cur_dlq_strt_dt, /*-- current delinquent start date*/
                         A.hi_bal, /*-- high balance*/
                         A.hi_bal_dt,  /*-- high balnace date*/
                         A.int_method, /*-- interest method*/
                         A.int_unclct_due_dt, /*-- interest only due date*/
                         A.int_only_pymt_freq, /*-- interest only payment frequency*/
                         A.int_only_pymt_typ, /*-- interest only payment type*/
                         A.int_pymt_inst_no, /*-- interest payment institution number*/
                         A.int_rt_or_prem, /*-- interest rate or premium*/
                         A.int_renew_dt, /*-- interest renewal date*/
                         A.int_rule, /*-- interest rule*/
                         A.int_trm_no_of_pymt, /*-- interest term - number of payments*/
                         A.iss_renw_dt, /*-- issue/renewal date*/
                         A.last_pymt_dt, /*-- last payment date*/
                         A.lst_pymt_waivd_dt, /*-- last  payment waived date*/
                         A.low_bal, /*-- low balance*/
                        A.low_bal_dt, /*-- low balance date*/
                        A.reneg_rduc_rt_ind, /*-- renegotiated reduced rate indicator*/
                        A.renegotiatn_dt, /*-- renegotiation date*/
                        A.tot_fees_clct, /*-- total fees collected*/
                        A.tot_int_clct, /*-- total interest collected*/
                        A.waived_pymt_cnt, /*-- waived payment count*/
                        A.wrt_off_amt,  /*-- write off amount*/
                        A.wrt_off_dt, /*-- write off date*/
                        A.wrt_off_exist_ind,  /*-- write off recovery amount total*/
                        A.wrt_off_rcvry_dt_last, /*-- write off last recovery date*/
                        A.wrt_off_rcvry_amt_tot, /*-- write off recovery amount total */
                        L.mif_srvc_id /*-- mif service id*/


                    FROM DDWV01.rb_clnt AS C	

                            /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                         INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                              ON  r.SNAP_DT  = c.snap_dt
                                  AND r.ORG_UNT_ID = c.resp_unt
                                  AND r.PARNT_HIER_TYP_ID = 80000 
                                  AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)

                         INNER JOIN DG6V19.bus_prtfol_lend as L
                             on L.mth_end_dt = C.snap_dt
                                and L.clnt_no = C.clnt_no
                                /*--and L.mth_end_dt LE '2018-05-31'*/
                                and L.mif_srvc_id = 20

                         INNER JOIN DG6V19.loan_hist as A
                             on A.mth_end_dt = L.mth_end_dt
                                and A.ar_id = L.ar_id 
                                and A.mif_srvc_id = L.mif_srvc_id 
                                and A.mif_srvc_id = 20
                                /*--and A.mth_end_dt LE '2018-05-31'*/

                    where clnt_grp NOT IN ('SB') and clnt_grp NOT IN ('OOO')
                          and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                          and c.clnt_typ=2 /* keep only non-personal clients */
                    and c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'
            """
    return pd.read_sql(qry, connection)

In [29]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_loan_hist(start_snap_dt, end_snap_dt)
df1.shape

(2451273, 39)

In [30]:
df1.to_csv(raw_data_dir+'loan_hist_y1.csv', index=False)

In [31]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_loan_hist(start_snap_dt, end_snap_dt)
df2.shape

(2469950, 39)

In [32]:
df2.to_csv(raw_data_dir+'loan_hist_y2.csv', index=False)

In [33]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_loan_hist(start_snap_dt, end_snap_dt)
df3.shape

(2504220, 39)

In [34]:
df3.to_csv(raw_data_dir+'loan_hist_y3.csv', index=False)

In [35]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_loan_hist(start_snap_dt, end_snap_dt)
df4.shape

(3769933, 39)

In [36]:
df4.to_csv(raw_data_dir+'loan_hist_y4.csv', index=False)

# Lend_General

In [4]:
def pull_lend_general(start_snap_dt, end_snap_dt):    
        qry = f"""\
                   select 
                    /* month end date */
                           C.SNAP_DT as mth_end_dt, 
                    /* client number */
                           C.clnt_no , 
                    /* client group, used to define commercial and small business clients */
                    case
                                when (c.BSC in (190,192,200,202,250,490,590)
                                        or (c.BSC in (310,500,502,504,505,510) and 
                                            (c.CURR_CMI_NO not in (100,101,102,103,
                                                                    200,201,202,203,
                                                                    300,301,302,303,
                                                                    401,402,403,
                                                                    501,502,503) or c.CURR_CMI_NO is NULL))
                                     ) then 'COMM'

                                when (c.BSC in (550,554,555)
                                        and (c.CURR_CMI_NO not in (100,101,102,103,
                                                                    200,201,202,203, 
                                                                    300,301,302,303,
                                                                    401,402,403,
                                                                    501,502,503) or c.CURR_CMI_NO is NULL)
                                     ) then 'AG'
                                when c.BSC in (330) then 'PUB'
                                when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                        and c.CURR_CMI_NO in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503)
                                     )then 'SB'
                                else 'OOO'
                            end as Clnt_Grp,
                             /* consolidation indicator added on 2019-01-23 */
                           C.CNSLN_NON_SURV_IND as cnsln_non_surv_ind,
                    /*arrangement id*/
                           L.ar_id ,   
                           'A' || trim(L.ar_id) as t_ar_id, /* added by QH on 2018-11-07 */
                    /*MIF service identifier,*/
                           L.mif_srvc_id,
                    /* product */
                           PD. product,
                    /*product open date*/
                           L.dt_opened as product_dt_opened, 
                    /* loan status */
                           L.ln_sts,
                    /* Loan Description*/ 
                           L.ln_desc , 
                    /* close reason code in RBDM - more reliable */
                           L.CL_REAS_CD as rbdm_cl_reas_cd, /*-- from performance monitoring*/
                    /* close reason code in EDW */
                           AR.CLS_REAS_CD as edw_cl_reas_cd,/* EDW close reason code, added on 2019-01-23 */
                           AR.OPEN_CLS_STS as edw_open_cl_sts, /* EDW open close status, added on 2019-01-23 */  
                           AR.DT_CLS as cls_dt, /* AR close date, added on 2019-01-23 */ 
                    /* write off info */
                           L.wrt_off_dt, /* added on 2019-02-01 */
                           L.wrt_off_exist_ind, /* added on 2019-02-01 */
                           L.wrt_off_reas, /* added on 2019-02-01 */
                           L.woff_to_dt_amt, /* added on 2019-02-01 */
                           L.woff_mtd_amt, /* added on 2019-02-01 */
                    /* Loan Type Code*/
                           L.ln_tp_cd,  
                    /* Interest Rule*/
                           L.int_rule,  
                    /* Month end balance*/ 
                           L.bal, 
                           LLL.av_bal_mth_len_cad as avg_bal,  /*-- This logic has been changed!!-> see the following*/
                           case 
                                when L.MIF_SRVC_ID in (20, 37, 63, 64) then zeroifnull(Cast(TL.MTH_AV_BAL_BDTE * CE.RT_VAL as decimal(15,2))) 
                                when L.MIF_SRVC_ID = 21 then Cast(MS.AVG_BAL_MTH * CE.RT_VAL as decimal(15,2)) 
                                when L.MIF_SRVC_ID = 23 then
                                                    zeroifnull(
                                                    case 
                                                        when VM.DLY_AVG_BAL_MTH <= 0 then 0 
                                                        when VM.VISA_PROD_CD in ('GUS', 'UXP') then Cast(VM.DLY_AVG_BAL_MTH * US.RT_VAL as decimal(15,2)) 
                                                        else VM.DLY_AVG_BAL_MTH
                                                    end)
                                when L.MIF_SRVC_ID in (25, 60) then L.BAL 
                                when L.MIF_SRVC_ID = 26 then Cast(TD.AVG_MTH_BAL_AMT * CE.RT_VAL as decimal(15,2)) 	
                            end as avg_bal_new,

                           L.mnemonic_desc,  /*--Detailed Loan description*/
                           L.fac_id, /*-- facility id*/
                           EXT.CARD_FAC_ID,
                           /*
                            Case
                                when (L.FAC_ID is null or L.FAC_ID = '') then '00000000000000000000'
                                else L.FAC_ID
                            End as FAC_ID,
                            */
                           L.dlqy_amt_31_60,               /*-- Amount being delinquent between 30 and 60 days*/
                           L.dlqy_amt_31_90,              /* --Amount being delinquent between 30 and 90 days*/
                           L.dlqy_amt_61_90,              /*--Amount being delinquent between 60 and 90 days*/
                           L.dlqy_amt_91_120,           /*-- Amount being delinquent between 90 and 120 days*/
                           L.dlqy_amt_121,                 /*-- Amount being delinquent for more than 120 days*/
                           L.tot_nal,                             /* -- Total non-accrual outstanding*/
                           L.nal_dlqy_amt_91,            /*--Amount non-accrual or delinquent more than 90 days*/
                           L.dlqy_strt_dt,               /*--Delinquency start date, added on 11-19-2020*/ 
                           L.tot_days_late,              /* added on 11-19-2020*/
                           L.tot_dlq,                    /* added on 11-19-2020*/

                           case
                             when LL.dw_srvc_id = 20 then LL.int_rt_or_prem
                             when LL.dw_srvc_id = 21 then LL.int_rt_or_prem * 100
                             else LL.int_rt_or_prem
                           end as client_pop, /*--interest rate or premium*/

                           LL.int_base_rate AS client_int_base_rate, /*--interest base rate*/
                           LL.int_rt_addn * 100 as int_rt_addn, /*--interest rate addition*/

                           LL.amort_trm_in_mth , /*-- ammortization term*/
                           LL.trm_in_mth , 
                           LL.cost_f_fnds_ntrst ,
                           LL.int_renw_dt_lst ,

                           case
                             when L.mif_srvc_id = 20 and L.int_rule in ('3' , '03', '53', '20') then  LL.int_base_rate
                             when L.mif_srvc_id = 20 and L.int_rule in ('4', '04', '9', '09', '10', '54') then  zeroifnull(LL.int_base_rate) + zeroifnull(LL.int_rt_or_prem) 
                             when L.mif_srvc_id = 21 then  zeroifnull(LL.int_rt_or_prem) * 100  + zeroifnull(LL.int_rt_addn) * 100
                             else 0 /*only mif_sid = 37 has Null interest rate and base and pop are zeros in dm */
                           end as client_rate, /*-- Client Interest Rate*/

                           V.cash_apr as visa_cash_apr, /*--cash APR*/
                           V.prch_apr as visa_prch_apr, /*-- visa APR*/
                           VV.cr_lmt /*-- credit limit*/


                    from DDWV01.RB_CLNT as C
                
                     /*Neo Added Logic */
                     INNER JOIN DG6V19.BUS_PRTFOL_CLNT AS BCP
                        ON  C.SNAP_DT  = BCP.MTH_END_DT
                        AND c.clnt_no = BCP.CLNT_NO                         
                        
                        /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                         INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                              ON  r.SNAP_DT  = c.snap_dt
                                  AND r.ORG_UNT_ID = c.resp_unt
                                  AND r.PARNT_HIER_TYP_ID = 80000 
                                  AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)

                        /* lending info in business portfolio lend table */
                          INNER JOIN DG6V19.bus_prtfol_lend as L
                             ON L.clnt_no = C.clnt_no
                                 and L.mth_end_dt = c.snap_dt
                            /*     --and L.mth_end_dt LE '2018-05-31'
                                 --and L.mth_end_dt >= '2016-05-31'*/

                          LEFT JOIN DDWV01.loan_info_tact as LL 
                               ON LL.ar_id = L.ar_id
                                  and LL.snap_dt = L.mth_end_dt
                                  and LL.dw_srvc_id = L.mif_srvc_id  /*-- added on 4th July, 2018, checkding by Kevin Ma
                                  */
                          LEFT JOIN DG6V19.bus_comm_rpt_lend_hist AS LLL /*min me_dt is 2011 Aug */
                              ON L.clnt_no = LLL.cd_primary_ipn
                                 and L.ar_id = LLL.prod_no
                                 and L.mif_srvc_id = LLL.mif_srvc_id
                                 and L.mth_end_dt = LLL.snap_dt
                                 /*--and LLL.snap_dt LE '2018-05-31'
                                 --and LLL.snap_dt >= '2016-05-31'*/

                          LEFT JOIN P3C.visa_mth_triadbill as V /*min me_dt 2004 Feb */
                              ON V.clnt_no = L.clnt_no
                                 and V.acct_no = L.ar_id
                                 and V.me_dt = L.mth_end_dt
                        /*		 --and V.me_dt LE '2018-05-31'
                                 --and V.me_dt >= '2016-05-31'	*/	 

                          LEFT JOIN DG6V19.VISA_NEW_HIST as VV 
                          ON VV.ar_id = L.ar_id
                                  and VV.mth_end_dt = L.mth_end_dt
                                  and VV.mif_srvc_id = L.mif_srvc_id


                            /*--The following tables are added in the monitoring SAS code*/
                            inner join DG6V19.ARNGMNT_OWN_HIST as AO
                                                 on AO.AR_ID = L.AR_ID
                                                and AO.CLNT_NO = L.CLNT_NO
                                                and AO.MTH_END_DT = L.MTH_END_DT
                                                and AO.MIF_SRVC_ID = L.MIF_SRVC_ID
                                                and AO.PRMRY_ALT_IND = 'P'
                                                and AO.MIF_SRVC_ID in (2,20,21,23,25,26,37,60,63,64) 

                            inner join DG6V19.ARNGMNT_HIST as AH
                                                 on AH.MIF_ACCT_NO = AO.MIF_ACCT_NO
                                                and AH.MIF_SRVC_ID = AO.MIF_SRVC_ID
                                                and AH.MTH_END_DT = AO.MTH_END_DT
                                                and AH.AR_ID = AO.AR_ID
                                                and AH.MIF_SRVC_ID in (2,20,21,23,25,26,37,60,63,64)
                            /* The following is added to extract the close date from AR on 2019-01-23*/
                            left join DDWV01.AR as AR
                                                on AO.AR_ID = AR.AR_ID
                                                and AO.MTH_END_DT = AR.SNAP_DT 
                                                and AO.MIF_SRVC_ID = AR.SRVC_ID

                            left join DG6V19.MORTG_MTHLY_STAT as MS
                                                 on MS.MIF_ACCT_NO = AH.MIF_ACCT_NO 
                                                and MS.MIF_SRVC_ID = AH.MIF_SRVC_ID  
                                                and MS.MTH_END_DT = AH.MTH_END_DT	
                                                and MS.MIF_SRVC_ID = 21

                            left join DG6V19.CRNCY_EXCH_RT as CE
                                                 on CE.CRNCY_CD = AH.CRNCY_CD
                                                and CE.MTH_END_DT = AH.MTH_END_DT 
                                                and CE.RT_TYP = '1'	

                            left join DG6V19.CRNCY_EXCH_RT as US
                                                 on US.MTH_END_DT = AH.MTH_END_DT 
                                                and US.CRNCY_CD = 'USD'
                                                and US.RT_TYP = '1'

                           left join DDWV01.TLN_LOANS_HIST as TL
                           on TL.AR_ID = L.AR_ID
                                and TL.SNAP_DT = L.MTH_END_DT
                                and TL.DW_SRVC_ID = L.MIF_SRVC_ID
                                and TL.DW_SRVC_ID in (20,37,63,64)

                           left join DG6V19.TRD_AR_HIST as TD
                           on TD.AR_ID = L.AR_ID
                                 and TD.MIF_SRVC_ID = L.MIF_SRVC_ID
                                 and TD.MTH_END_DT = L.MTH_END_DT
                                 and TD.MIF_SRVC_ID = 26


                            left join P3C.VISA_ACCT_MTH_ST_R as VM
                            on CAST(VM.ACCT_NO as FORMAT'9(20)') = L.AR_ID
                                  and VM.ME_DT = L.MTH_END_DT     
                                  and VM.SRVC_ID = 22
                                  and L.MIF_SRVC_ID = 23

                            left join DL_Fortis.fortis_prd_hist as PD
                            on    C.CLNT_NO = PD.CLNT_NO and /* This was added to reduce duplicates QH, 2018-09-10 */
                                  L.AR_ID = PD.ar_id and
                                  c.snap_dt = PD.mth_end_dt
                                  and L.MIF_SRVC_ID = PD.MIF_SRVC_ID /* This was added to reduce duplicates QH, 2018-09-10 */

                            /* The following is to add facility id for MIF_SRVC_ID = 23(visa) */
                            left join DL_EXT.dl_LEND_EXT as EXT
                            on EXT.clnt_no = C.clnt_no
                               and EXT.mth_end_dt = c.snap_dt
                               and EXT.MIF_SRVC_ID = L.MIF_SRVC_ID
                               and L.MIF_SRVC_ID = 23
                               and EXT.AR_ID = L.AR_ID

                    where 
                          clnt_grp NOT IN ('SB') and clnt_grp NOT IN ('OOO')
                          AND TOT_SN_EXP > 1000000
                          and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                          and c.clnt_typ=2 /* keep only non-personal clients */                  
                          and c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'
            """
        return pd.read_sql(qry, connection)

In [5]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_lend_general(start_snap_dt, end_snap_dt)
df1.shape

(4511262, 49)

In [6]:
df1.to_csv(raw_data_dir+'lend_general_y1.csv', index=False)

In [7]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_lend_general(start_snap_dt, end_snap_dt)
df2.shape

(4600664, 49)

In [8]:
df2.to_csv(raw_data_dir+'lend_general_y2.csv', index=False)

In [9]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_lend_general(start_snap_dt, end_snap_dt)
df3.shape

(4591727, 49)

In [10]:
df3.to_csv(raw_data_dir+'lend_general_y3.csv', index=False)

In [11]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_lend_general(start_snap_dt, end_snap_dt)
df4.shape

(7307761, 49)

In [12]:
df4.to_csv(raw_data_dir+'lend_general_y4.csv', index=False)

# Financials

In [ ]:
inancial_qry = f"""\
    SELECT 
     /*-- Snap Shot Date*/
    F.SNAP_DT,
    /*-- Client SRF - an internal RBC Client Identification Number*/
    F.CLNT_NO,        
    /* client group, used to define commercial and small business clients */
    case
                when (c.BSC in (190,192,200,202,250,490,590)
                        or (c.BSC in (310,500,502,504,505,510) and 
                            (c.CURR_CMI_NO not in (100,101,102,103,
                                                    200,201,202,203, 
                                                    300,301,302,303,
                                                    401,402,403,
                                                    501,502,503) or c.CURR_CMI_NO is NULL))
                     ) then 'COMM'

                when (c.BSC in (550,554,555)
                        and (c.CURR_CMI_NO not in (100,101,102,103,
                                                    200,201,202,203,
                                                    300,301,302,303,
                                                    401,402,403,
                                                    501,502,503) or c.CURR_CMI_NO is NULL)
                     ) then 'AG'
                when c.BSC in (330) then 'PUB'
                when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                        and c.CURR_CMI_NO in (100,101,102,103,
                                                200,201,202,203, 
                                                300,301,302,303,
                                                401,402,403,
                                                501,502,503)
                     )then 'SB'
                else 'OOO'
            end as Clnt_Grp,
    /*-- A unique statement ID for each financial statement.*/
    F.STMT_ID,  
    /*-- System Source ID*/
    F.SYS_SRC_ID,    

    /* --  The sum of all expenses on the income statement that have not been paid or logged under accounts payable during an accounting period; 
     --  in other words, obligations for goods and services provided to a company for which invoices have not yet been received. */
     TOT_ACCR_LBLTY_AMT, 

    /* -- A trade payable is an amount billed to a company by its suppliers for goods delivered to or services consumed by the company in the ordinary course of business. 
     --Trade payables are recorded in a separate accounts payable account, with a credit to the accounts payable account and a debit to whichever account most closely 
     -- represents the nature of the payment, such as an expense or an asset.*/
     AP_TRD_AMT,    

    /*-- Other accounts receivable are financial assets consisting of non-trade advances and other items due to be received.*/
     AR_OTH_AMT,   

    /*  -- Trade receivables are amounts billed by a business to its customers when it delivers goods or services to them in the ordinary course of business. 
      -- These billings are typically documented on formal invoices, which are summarized in an accounts receivable aging report.*/
     AR_TRD_AMT, 

    /*-- The cumulative depreciation of an asset up to a single point in its life. 
    -- Regardless of the method used to calculate it, the depreciation of an asset during a single period is added to the previous period's accumulated depreciation 
    -- to get the current accumulated depreciation.*/
     ACCUM_DEPR_AMT,  

    /*-- The amount of money that a company has left over after all corporate income and withholding taxes have been deducted from taxable income. 
    -- After-tax income represents the amount of disposable income that a firm has to spend on future investments or on present consumption.*/
     AFT_TX_INCM_AMT, 

    /*-- Amortization expense is the write-off of an asset over its expected period of use, which reflects the consumption of the asset.*/
     AMORT_EXPNC_AMT,  

    /*-- "Audit Method John - French & English mixed.  use ref code?  TBD"*/
     ADT_MTHD_NM,  

    /*-Cash is the most liquid asset a company can own. A company's cash account includes all currency and coins owned by the company as well as all 
      -- deposits in the bank including checking accounts and savings accounts. Cash also includes instruments or contracts that can be deposited in a bank account like vendee checks, 
      -- customer checks, cashier's checks, certified checks, as well as money orders.*/
     CSH_AMT,  

     /*-- Whether depreciation is included in cost of goods sold or in operating expenses depends on the type of asset being depreciated. 
       -- Depreciation is listed with cost of goods sold if the expense associated with the fixed asset is used in the direct production of inventory.*/
     COGS_DEPR_AMT,  

    /*-- This is a long term asset account that records the accumulated cost of a project (typically, applied to capital budget items) that has not yet been completed. 
      --When the project is finished and placed into the service, the cost is removed from this account and is recorded in a related long-term asset account. 
     --This asset item is not depreciated until the asset is placed in service. Normally, upon completion, a CIP item is reclassified, and the reclassified asset is capitalized and depreciated.*/
     CONST_IN_PRGRS_AMT, 

    /*-- The direct costs attributable to the production of the goods sold by a company. 
       -- This amount includes the cost of the materials used in creating the good along with the direct labor costs used to produce the good.*/
     COGS_AMT,  

    /*--COST IN EXCESS OF BILLINGS, in percentage of completion method, is when the billings on uncompleted contracts are less than the income earned to date.*/
     CST_IN_EXCS_OF_BL_AMT, 

     /*--A portion of the balance sheet that represents the total amount of long-term debt that must be paid within the next year. */
     CURR_POR_OF_LT_DEBT_AMT, 

    /*--Subordinated debts are repayable after other debts have been paid, meaning they are more risky for the lender of the money. The debts may be secured or unsecured. Subordinated debt typically has a lower credit rating, and, therefore, a higher yield than senior debt.*/
     CURR_POR_OF_SUB_DEBT_AMT, 

    /*--Depreciation expense is the allocated portion of the cost of a company's fixed assets that is appropriate for the accounting period indicated on the company's income statement.*/
     DEPR_EXPNC_AMT, 

    /*-- Dividends payable are dividends that a company's board of directors has declared to be payable to its shareholders. Until such time as the company actually pays the shareholders, the cash amount of the dividend is recorded within a dividends payable account as a current liability.*/
     DVDND_PYBL_AMT, 

     /*--The aggregate amount of receivables to be collected from related parties where one party can exercise control or significant influence over another party, including affiliates, owners or officers and their immediate families, pension trusts, and so forth, at the financial statement date. These receivables are usually due within one year (or one business cycle).*/
     DUE_TO_REL_PRTY_AMT, 

     /*-- Dividends on an ESOP - a retirement plan that is designed to provide employees with an ownership interest in the company for which they work by investing primarily in stock of the employer.*/
     ESOP_DVDND_AMT, 

     /*--Finished goods are goods that have completed the manufacturing process but have not yet been sold or distributed to the end user.*/
     FINSHD_GDS_INVTY_AMT, 

    /* --Assets that are purchased for long-term use and are not likely to be converted quickly into cash, such as land, buildings, and equipment.*/
     FIX_AST_AMT, 

     /*-- The amount of income tax that is associated with (matches) the net income reported on the company's income statement. This amount will likely be different than the income taxes actually payable, since some of the revenues and expenses reported on the tax return will be different from the amounts on the income statement.*/
     INCM_TX_EXPNC_AMT, 

    /*-- Income tax receivable is money a company expects from tax revenue agencies. */
     INCM_TX_RCV_AMT, 

    /*-- Interest expense is the cost of debt that has occurred during a specified period of time.*/
     INT_EXPNC_AMT, 

    /*-- The term that companies use on their income statement for reporting the interest earned on cash temporarily held in savings accounts, certificates of deposits, or other investments. Because the interest wasn't part of the original investment, they record it separately, as interest income.*/
     INT_INCM_AMT, 

    /*-- A long-term asset account that reports the cost of real property exclusive of the cost of any constructed assets on the property. Land usually appears as the first item under the balance sheet heading of Property, Plant and Equipment. Generally, land is not depreciated.*/
     LAND_AMT, 

    /* -- "The expense recorded when the company does not have the rights of ownership of an asset but has been allowed the use of an asset. An operating lease is not capitalized; it is accounted for as a rental expense in what is known as ""off balance sheet financing."" */
     LSE_RENT_EXPNC_AMT, 

     /*-- The LIFO reserve is the difference between the accounting cost of an inventory that is calculated using the FIFO method, and one using the LIFO method. */
     LIFO_RES_AMT, 

     /*-- Loans and financial obligations lasting over one year. Long-term debt for a company would include any financing or leasing obligations that are to come due in a greater than 12-month period. Such obligations would include company bond issues or long-term leases that have been capitalized on a firm's balance sheet.*/
     LT_DEBT_AMT, 

    /* --"Marketable securities are unrestricted financial instruments which can be readily sold on a stock exchange or bond exchange. Marketable securities are often classified into two groups: marketable equity securities and marketable debt securities."*/
     MRK_SCTY_AMT,  

     /*-- Net operating profit refers to the amount of money that a company has earned after the cost of goods sold and operating expenses have been deducted.*/
     NET_OPER_PROF_AMT, 
     /*-- Net sales are operating revenues earned by a company for selling its products or rendering its services. Also referred to as revenue, they are reported directly on the income statement as Sales or Net sales*/
     NET_SALE_AMT,  
     /*-- A non-operating asset is an asset that generates income, but is unrelated to the core operations of the company.*/
     NON_OPER_CURR_AST_AMT, 
     /*-- "Accumulated other comprehensive income is a subsection in equity where ""other comprehensive income"" is accumulated (summed or ""aggregated""). The balance of AOCI is presented in the Equity section of the Balance Sheet as is the Retained Earnings balance, which aggregates past and current Earnings, and past and current Dividends.".*/
     OTH_COMP_INCM_AMT, 
     OTH_INVTY_AMT,
     /*-- Revenue generating assets that have not been documented elsewhere as an Operating Asset.*/
     OTH_OPER_AST_AMT, 
    /*-- An overdraft occurs when money is withdrawn from a bank account and the available balance goes below zero. In this situation the account is said to be "overdrawn". */
     OD_AMT,   
     /*-- "A profitability measure that looks at a company's profits before the company has to pay corporate income tax. This measure deducts all expenses from revenue including interest expenses and operating expenses, but it leaves out the payment of tax. "*/
     PROF_BEF_TX_AMT, 
     /*-- "A company's stock of the basic material from which a product is manufactured."*/
     RAW_MATL_INVTY_AMT, 
     /*--"Reported on the income statement, it is the sum of all direct and indirect selling expenses and all general and administrative expenses of a company. "*/
     SELL_AND_GNRL_EXPNC_AMT, 
     /*-- Source currency code.*/
     SRC_CRNCY_CD, 
     /*-- The end date of the accounting period for which the financial statements were prepared.*/
     STMT_DT,  
    /*-- Denotes the number of months in the accounting period captured by the financial statement.*/
     STMT_MTH_CNT,  
    /*-- "A loan scheduled to be repaid in less than a year "*/
     SHRT_TRM_LN_PYBL_AMT,  
     /*-- Debt which ranks after other debts if a company falls into liquidation or bankruptcy.*/
     SUB_DEBT_LBLTY_AMT, 
    /*  -- Work in Progress???.*/
     WIP_INVTY_AMT, 
    /*-- Trade receivables (Net) are all accounts from trade, net of allowance for doutful accounts.*/
     NET_TRD_AR_AMT,  
    /*-- The amount by which assets exceed liabilities.*/
     TOT_NET_WORTH_AMT, 
    /*-- A balance sheet account that represents the value of all assets that are reasonably expected to be converted to cash within one year in the normal course of business.*/
     TOT_CURR_AST_AMT, 
     /*-- A company's debts or obligations that are due within one year. Current Liabilities appears on the balance sheet and include short term debt, accounts payable, accrued liabilities, and other debt.*/
     TOT_CURR_LBLTY_AMT,   
     /*-- The aggregate of all debts a company is liable for. Total Liabilities is calculated by summing all short term and long term liabilities.*/
     TOT_LBLTY_AMT,   
      /*-- The direct costs attributable to the production of goods sold by a company. Includes cost of materials and direct labour costs used to produce the good. It excludes indirect expenses. It appears on the income statement and is deducted from revenue to calculate gross margin. Also referred to as "cost of sales."*/
     TOT_COGS_AMT, 
     /*-- A companies debt, such as bonds, long-term notes payable, or debentures that will mature in more than one year or one business cycle. Classified as funded debt as it is funded by interest payments over the term of the loan. Synonymous with long term debt.*/
     NET_FND_DEBT_AMT,  
     /*-- Debt service coverage ratio (DSCR) also known as the debt coverage ratio (DCR) is the ratio of cash available for debt servicing for interest, principal, and lease payments. It is calculated as Net Operating Income/Debt Services. It is a popular benchmark used in the measurement of an entity's ability to produce enough cash to cover its debt (including lease payments).*/
     DEBT_SRVC_CVRG_RATO,  
     /*-- Investment securities that are short term, have high credit quality, and are highly liquid. They include government treasury bills, bank certificates of deposit, corporate commercial paper, and other money market instruments.*/
     CSH_AND_EQUIV_AMT,  
     /*-- "Identifies the accounting standard or generally accepted account principles (GAAP) applied during the preparation of the financial statements. "*/
     ACCTG_STD_NM,  
    /*-- The sum of all raw materials, work-in-process goods, and completely finished goods that are considered to be the portion of a business's assets that are ready or will be ready for sale.*/
     TOT_INVTY_AMT,  
    /*-- Gross Fixed Assets is the purchase price of all fixed assets (Land, buildings, equipment, machinery, vehicles, leasehold improvements) before taking into account the accumulated depreciation or consumption of the fixed assets.*/
     GRS_FIX_AST_AMT,  
     /*-- Net Fixed Assets is the purchase price of all fixed assets (Land, buildings, equipment, machinery, vehicles, leasehold improvements) less accumulated Depreciation.*/

     NET_FIX_AST_AMT,  
     /*-- A company's long term investments, where the full value will not be realized within the accounting year. Noncurrent assets are capitalized rather than expensed, meaning that the company allocates the cost of the asset over a number of years for which the asset will be in use, instead of allocating the entire cost to the accounting year in which the asset was purchased.*/
     TOT_NON_CURR_AST_AMT,  
     /*-- A company's long term financial obligations that are not due within the present accounting year. Examples are long-term borrowing, bonds payable, and long-term lease obligations.*/
     TOT_NON_CURR_LBLTY_AMT, 
     /*-- A measure of a company's financial leverage calculated by dividing its total liabilities by stockholders equity. It indicates what proportion of equity the company is using to finance its assets. Also called Debt/Equity Ratio.*/
     TOT_LBLTY_NET_WORTH_AMT,  
     /*-- A measure of overall sales that is not adjusted for customer discounts or returns, calculated by simply adding all sales invoices, and not including operating expenses, cost of goods sold, payment of taxes, or any other charge.*/
     GRS_SALE_AMT,  
     /*-- A company's revenue minus its cost of goods sold. Gross profit is a company's residual profit after selling a product/service and deducting the cost associated with its production and sale. Also called gross margin.*/
     GRS_PROF_AMT,  
     /*-- Those expenditures that a business incurs to engage in any activities not directly associated with the production of goods or services.*/
     OPER_EXPNC_AMT,  
     /*-- The financial payments and non monetary benefits provided to high level management in exchange for their work on behalf of an organization.*/
     OFFCR_COMPEN_AMT, 
     /*-- The sum of operating expenses which are any expenses associated with the general, sales, and administrative functions of an entity.*/
     TOT_OPER_EXPNC_AMT,  
     /*-- Expenses incurred related to the income earned from activities other than normal business operations, such as investment interest, foreign exchange gains (losses), and profit (loss) from the sale of non-inventory assets.*/
     TOT_OTH_INCM_EXPNC_AMT,  
     /*--The amount of income tax that is associated with (matches) the net income reported on the company's income statement. This amount will likely be different than the income taxes actually payable, since some of the revenue and expenses on the tax return will be different from the amounts on the income statement.*/
     NET_INCM_TX_EXPNC_AMT,   
     /*--This represents net profit before being adjusted for extraordinary items. Extraordinary items are gains and losses which are infrequent and atypical in nature. They are accounted for separately so they don't skew the company's regular earnings.*/
    PROF_BEF_EXTRORD_ITM_AMT,   

    /*--Often referred to as the bottom line, net profit is calculated by subtracting a company's total expenses from total revenue, thus showing what a company has earned (or lost) in a given period of time.*/
    NET_PROF_AMT,   
    /*--Comprehensive income is the change in equity of a business enterprise during a period from transactions and other events from non-owner sources. It includes all non-owner changes in equity (in contrast to net income which does not include some changes in equity).*/
    COMP_INCM_AMT,   
    /*--The capital of a business that is used in its day-to-day trading operations, calculated as the current assets minus the current liabilities.*/
    WRK_CAPTL_AMT,   
    /*--An indicator of a company's short-term liquidity. The quick ratio measures a company's ability to meet its short-term obligations with its most liquid assets. */
    QUICK_RATO,   
     /*--A liquidity ratio that measures a company's ability to pay short-term obligations.*/
    CURR_RATO,  
    /*--A measure of the physical worth of a company, which does not include any value derived from intangible assets such as copyrights, patents and intellectual property. Tangible net worth is calculated by taking a firm's total assets and subtracting the value of all liabilities and intangible assets.*/
    TANGBL_NET_WORTH_AMT, 
    /*--Debt to tangible net worth is a derivative of the debt-to-equity ratio. */
    DEBT_TO_TOT_NET_WORTH_RATO,   
    /*--The liabilities to assets ratio is a solvency ratio that examines how much of a company's assets are financed by liabilities.*/
    LIABS_TO_AST_RATO, 
     /*--A ratio used to determine how easily a company can pay interest on outstanding debt from its earnings before interest and taxes. */
    INT_CVRG_RATO,  
    /*--A measure of the earnings of a company that adds the interest expense, depreciation and amortization back to the net income number, but takes the tax expense into consideration.*/
    EBIDA_AMT,  
    /*--The fixed charge coverage ratio measures a firm's ability to pay all of its fixed charges or expenses with its income before interest and income taxes.*/
    FIX_CHRG_CVR_AMT,  
     /*--An indicator of how profitable a company is relative to its total assets.*/
    RET_ON_AST_RATO,  
    /*--Provides the rate of return to the company's shareholders. Calculated as net income as a percentage of shareholders equity.*/
    RET_ON_EQTY_PCT,   
     /*--A company's total sales revenue minus its cost of goods sold, divided by the total sales revenue, expressed as a percentage.*/
    GRS_MRGN_PCT,  
    /*--A ratio used to measure a company's pricing strategy and operating efficiency. A measurement of what proportion of a company's revenue is left over after paying for variable costs of production such as wages, raw materials, etc.*/
    OPER_PROF_MRGN_RATO,   
    /*--A ratio of profitability calculated as net income divided by revenues, or net profits divided by sales. */
    PROF_MRGN_RATO,   
    /*--The number of days that it would take for the company to collect all amounts due from customers (before deducting doubtful accounts). The point of the measurement is to determine the effectiveness of a company's credit and collection efforts in allowing credit to reputable customers, as well as its ability to collect cash from them. For example, if the credit terms are net 30 days, you would expect this to be at least 30 days.*/
    GRS_ACCT_RCV_DYS_CNT,   
     /*--The number of days that it would take for the company to collect all amounts due from customers (net of doubtful accounts). The point of the measurement is to determine the effectiveness of a company's credit and collection efforts in allowing credit to reputable customers, as well as its ability to collect cash from them. For example, if the credit terms are net 30 days, you would expect this to be at least 30 days.*/
    NET_ACCT_RCV_DYS_CNT,  
    /*--A measure of a company's performance that gives investors an idea of how long it takes a company to turn its raw material into sales.*/
    RAW_MATL_INVTY_DYS_CNT,   
    /*--A measure of a company's performance that gives investors an idea of how long it takes a company to turn its work in progress into sales.*/
    WRK_IN_PRGRS_INVTY_DYS_CNT,   
     /*--A measure of a company's performance that gives investors an idea of how long it takes a company to turn its finished goods into sales.*/
    FINSHD_GDS_INVTY_DYS_CNT,  
    /*--The number of days it takes a company to turn over its inventory.*/
    TOT_INVTY_DYS_CNT,   
    /*--A company's average payable period. Days payable outstanding tells how long it takes a company to pay its invoices from trade creditors, such as suppliers.*/
    ACCT_PYBL_DYS_CNT,   
     /*--The change in total assets over completed fiscal periods.*/
    TOT_AST_GRWTH_PCT,  
    /*--The change in total liabilities over completed fiscal periods.*/
    TOT_LBLTY_GRWTH_PCT,   
     /*--The change in net worth over completed fiscal periods.*/
    NET_WORTH_GRWTH_PCT,  
    /*--The change in operating profit over completed fiscal periods.*/
    OPER_PROF_GRWTH_PCT,   
     /*--The change in net profit over completed fiscal periods.*/
    NET_PROF_GRWTH_PCT,  
     /*--A lease considered a purchased asset for accounting purposes in contrast to an operating lease which is a rental.*/
    CAPTL_LSE_OBLGN_AMT,  
    /*--Estimates the ability of a company to use its cash to generate sales, eg. through purchasing inventory. Typically expressed as a ratio of working capital to sales.*/
    SALE_TO_WRK_CAPTL_RATO,   
     /*--Ratio of sales to fixed assets measuring a company's ability to generate sales from fixed asset investments, eg. property, plant, and equipment.*/
    SALE_TO_FIX_AST_RATO,  
     /*--The amount of physical assets a company has net of its liabilities. It represents the supposed liquidation proceeds if operations ceased immediately.*/
    EFF_TANGBL_NET_WORTH_AMT,  
    /*--Also called Debt/Equity ratio. A measure of a company's financial leverage calculated by dividing its total liabilities by shareholder's equity.*/
    DEBT_TO_WORTH_RATO,   
    /*--Also called Asset Turnover Ratio. The amount of sales or revenues generated per dollar of assets. The asset turnover ratio is a indicator of the efficiency with which a company is deploying its assets. Calculated as Asset Turnover= Sales or Revenue/Total Assets*/
    SALE_TO_TOT_AST_RATO, 
     /*--Measures how well a company uses equity from its stockholders to generate sales. Calculated as Sales Revenue/Net Worth. Also called Sales to Equity Ratio.*/
    SALE_TO_NET_WORTH_RATO,  
    /*--The sum of current and long-term assets owned by the company.*/
    TOT_AST_AMT,   
    /*--An accounting measure calculated using a company's net earnings, before interest expenses, taxes, depreciation and amortization are subtracted, as a proxy for a company's current operating profitability, i.e., how much profit it makes with its present assets and its operations on the products it produces and sells, as well as providing a proxy for cash flow.*/
    EBITDA_AMT,   
     /*--Consolidated financial statements are the financial statements of a group in which the assets, liabilities, equity, income, expenses and cash flows of the parent (company) and its subsidiaries are presented as those of a single economic entity, according to International Accounting Standard John - perhaps replace with more meaningful Y or N"*/
    CNSLN_IND,  
    /*--Should data flow from MRA to FACT - Y/N*/
    FACT_IND,   
     /*--Access Group*/
    ACS_GRP_NM,  
    /*--The level of reporting i.e. Audit, Review &  Compilation, etc*/
    STMT_TYP_NM,   
     /*--Expenses on the income statement that have not been paid or logged under accounts payable during an accounting period; in other words, obligations for goods and services provided to a company for which invoices have not yet been received. */
    ACCR_LBLTY_AMT,  
    /*--Interest payable is the interest expense that has been incurred (has already occurred) but has not been paid as of the date of the balance sheet. (The interest payable amount does not include the interest for the periods of time which follow the date of the balance sheet.)*/
    INT_PYBL_AMT,   
    /*--Ratio of EBITDA to Interest Expense.*/
    EBITDA_TO_INT_EXPNC_RATO,   
     /*--A non-current liability that can be found on a parent company's balance sheet that represents the proportion of its subsidiaries owned by minority shareholders."*/
    MNRTY_INT_EXPNC_AMT,  
    /*--Amounts owing from related companies.*/
    AR_REL_CO_AMT,   
    F.LST_UPDT_DT_TM,
    /*--Net sales is equal to gross sales, minus discounts, customer returns, and allowances for defective/damaged merchandise. Net sales growth from one financial period to another is compared by the equation (Current period net sales- Prior period net sales)/ Prior period net sales * 100.*/
    NET_SALE_GRWTH_RATO,   
    /*--An account created in the income statement section of a business' financial statements that holds a suitable amount of funds meant to pay off upcoming interest payments. Furthermore, this type of interest is seen as an asset and unlike most conventional types of interest, it also is expensed over time.*/
    CPTLZ_INT_AMT,   
    /**********************CCRI selected Financial Module Variables listed below********************************/
    /*-- Net sales to Account Receivables: It measures how quickly the client is paid by its customers. 
    -- The lower the value the longer time it'll take for customers to pay, and longer period of exposure to higher delinquency possibility*/ 
    (Cast(net_sale_amt AS DECIMAL(25,6))/NullIfZero(net_trd_ar_amt))  AS ccri_Net_Sales_to_AR,  

    /*-- Cost of goods sold to Accounts Payables: It measures how quickly the client pays its suppliers. The lower the value
    -- the longer time the clients needs to  pay back its suppliers which indicates the worsening financial situation*/
    ( Cast(tot_cogs_amt AS DECIMAL(25,6))/NullIfZero(AP_TRD_AMT))  AS ccri_Cogs_to_AP,  

    /*-- Net funded debt to EBITDA: It measures how quickly the client pays back its debt given the stable operation of the business
    -- The higher the value, the longer period it'll take to payback the debt*/
    (Cast(net_fnd_debt_amt AS DECIMAL(25,6))/NullIfZero(ebitda_amt))  AS ccri_Net_fnd_debt_to_EBITDA , 

    /*-- Leverage ratio: It measures the capital structure of a company. The higher the value, more equity share in the capital structure
    -- the less burdened by interest payment*/																																										   
    (Cast(TOT_NET_WORTH_AMT AS DECIMAL(25,6))/NullIfZero(TOT_LBLTY_AMT))  AS ccri_leverage, 

    /*-- Debt service coverage ratio from MRA directly*/
    DEBT_SRVC_CVRG_RATO  AS ccri_DSCR,   
    /*-- Debt service coverage ratio derived*/
    Cast(EBITDA_AMT AS DECIMAL(25,6))/NullIfZero( Cast(INT_EXPNC_AMT AS DECIMAL(25,6))+Cast(CURR_POR_OF_LT_DEBT_AMT AS DECIMAL(25,6)))  AS ccri_DSCR_derived, 

    /*-- Growth of net sales*/
    NET_SALE_GRWTH_RATO AS ccri_net_sales_growth, 
    /*-- Cash and marketable securities to total assets*/
    (Cast(CSH_AMT AS DECIMAL(25,6))+Cast(MRK_SCTY_AMT AS DECIMAL(25,6)))/NullIfZero(tot_ast_amt)  AS ccri_csh_mktscty_to_ast, 
    /*--  Current ratio from MRA directly*/
    CURR_RATO AS ccri_curr_ratio, 
    /*-- Current ratio derived using the formula*/
    (Cast(TOT_CURR_AST_AMT AS DECIMAL(25,6))/NullIfZero(TOT_CURR_LBLTY_AMT)) AS ccri_curr_ratio_derived, 
    /*-- EBITDA margin*/
    (Cast(EBITDA_AMT AS DECIMAL(25,6))/NullIfZero(NET_SALE_AMT)) AS EBITDA_margin, 

    /**********************Other potential Financial Variables listed below********************************/
    /*--Net sales to Total Assets*/
    Cast(net_sale_amt AS DECIMAL(25,6))/NullIfZero(tot_ast_amt) as net_sales_to_tot_assets  ,  
    /*-- Assets to Libability*/
    Cast(tot_ast_amt AS DECIMAL(25,6))/NullIfZero(TOT_LBLTY_AMT) as assets_to_liability , 
    /*-- EBITDA to interest Expense Ratio*/
    /*EBITDA_TO_INT_EXPNC_RATO  , */
    /*-- Profit before tax to Total Assets*/
    Cast(PROF_BEF_TX_AMT AS DECIMAL(25,6))/NullIfZero(tot_ast_amt) as profit_btax_to_tot_assets 
    /*-- Total Asset amount,*/
    /*tot_ast_amt, */
    /*-- Net sales amount*/
    /*NET_SALE_AMT */

    FROM DG6V05.RSK_ANAL_STMT as F
        INNER JOIN 
             (SELECT CLNT_NO, SNAP_DT, max(STMT_DT) as max_STMT_DT
               FROM DG6V05.RSK_ANAL_STMT 
               GROUP BY CLNT_NO, SNAP_DT
               WHERE STMT_DT <= SNAP_DT) as FF
        ON F.CLNT_NO = FF.CLNT_NO and F.SNAP_DT = FF.SNAP_DT and FF.max_STMT_DT = F.STMT_DT

        INNER JOIN  DDWV01.rb_clnt AS C
        ON F.CLNT_NO = C. CLNT_NO
                and F.SNAP_DT = C.snap_DT

        /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
         INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
              ON  r.SNAP_DT  = c.snap_dt
                  AND r.ORG_UNT_ID = c.resp_unt
                  AND r.PARNT_HIER_TYP_ID = 80000 
                  AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)


    WHERE C.snap_DT > '{start_snap_dt}' AND C.snap_DT < '{end_snap_dt}'
    and   clnt_grp NOT IN ('SB') and clnt_grp NOT IN ('OOO')
          and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
          and c.clnt_typ=2 /* keep only non-personal clients */
    ORDER BY F.CLNT_NO, F.SNAP_DT, F.STMT_DT

In [46]:
def pull_financials(start_snap_dt, end_snap_dt):
    qry = f"""\
                SELECT 
                 /*-- Snap Shot Date*/
                F.SNAP_DT,
                /*-- Client SRF - an internal RBC Client Identification Number*/
                F.CLNT_NO,        
                /* client group, used to define commercial and small business clients */
                case
                            when (c.BSC in (190,192,200,202,250,490,590)
                                    or (c.BSC in (310,500,502,504,505,510) and 
                                        (c.CURR_CMI_NO not in (100,101,102,103,
                                                                200,201,202,203, 
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503) or c.CURR_CMI_NO is NULL))
                                 ) then 'COMM'

                            when (c.BSC in (550,554,555)
                                    and (c.CURR_CMI_NO not in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503) or c.CURR_CMI_NO is NULL)
                                 ) then 'AG'
                            when c.BSC in (330) then 'PUB'
                            when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                    and c.CURR_CMI_NO in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503)
                                 )then 'SB'
                            else 'OOO'
                        end as Clnt_Grp,
                /*-- A unique statement ID for each financial statement.*/
                F.STMT_ID,  
                /*-- System Source ID*/
                F.SYS_SRC_ID,    

                /* --  The sum of all expenses on the income statement that have not been paid or logged under accounts payable during an accounting period; 
                 --  in other words, obligations for goods and services provided to a company for which invoices have not yet been received. */
                 TOT_ACCR_LBLTY_AMT, 

                /* -- A trade payable is an amount billed to a company by its suppliers for goods delivered to or services consumed by the company in the ordinary course of business. 
                 --Trade payables are recorded in a separate accounts payable account, with a credit to the accounts payable account and a debit to whichever account most closely 
                 -- represents the nature of the payment, such as an expense or an asset.*/
                 AP_TRD_AMT,    

                /*-- Other accounts receivable are financial assets consisting of non-trade advances and other items due to be received.*/
                 AR_OTH_AMT,   

                /*  -- Trade receivables are amounts billed by a business to its customers when it delivers goods or services to them in the ordinary course of business. 
                  -- These billings are typically documented on formal invoices, which are summarized in an accounts receivable aging report.*/
                 AR_TRD_AMT, 

                /*-- The cumulative depreciation of an asset up to a single point in its life. 
                -- Regardless of the method used to calculate it, the depreciation of an asset during a single period is added to the previous period's accumulated depreciation 
                -- to get the current accumulated depreciation.*/
                 ACCUM_DEPR_AMT,  

                /*-- The amount of money that a company has left over after all corporate income and withholding taxes have been deducted from taxable income. 
                -- After-tax income represents the amount of disposable income that a firm has to spend on future investments or on present consumption.*/
                 AFT_TX_INCM_AMT, 

                /*-- Amortization expense is the write-off of an asset over its expected period of use, which reflects the consumption of the asset.*/
                 AMORT_EXPNC_AMT,  

                /*-- "Audit Method John - French & English mixed.  use ref code?  TBD"*/
                 ADT_MTHD_NM,  

                /*-Cash is the most liquid asset a company can own. A company's cash account includes all currency and coins owned by the company as well as all 
                  -- deposits in the bank including checking accounts and savings accounts. Cash also includes instruments or contracts that can be deposited in a bank account like vendee checks, 
                  -- customer checks, cashier's checks, certified checks, as well as money orders.*/
                 CSH_AMT,  

                 /*-- Whether depreciation is included in cost of goods sold or in operating expenses depends on the type of asset being depreciated. 
                   -- Depreciation is listed with cost of goods sold if the expense associated with the fixed asset is used in the direct production of inventory.*/
                 COGS_DEPR_AMT,  

                /*-- This is a long term asset account that records the accumulated cost of a project (typically, applied to capital budget items) that has not yet been completed. 
                  --When the project is finished and placed into the service, the cost is removed from this account and is recorded in a related long-term asset account. 
                 --This asset item is not depreciated until the asset is placed in service. Normally, upon completion, a CIP item is reclassified, and the reclassified asset is capitalized and depreciated.*/
                 CONST_IN_PRGRS_AMT, 

                /*-- The direct costs attributable to the production of the goods sold by a company. 
                   -- This amount includes the cost of the materials used in creating the good along with the direct labor costs used to produce the good.*/
                 COGS_AMT,  

                /*--COST IN EXCESS OF BILLINGS, in percentage of completion method, is when the billings on uncompleted contracts are less than the income earned to date.*/
                 CST_IN_EXCS_OF_BL_AMT, 

                 /*--A portion of the balance sheet that represents the total amount of long-term debt that must be paid within the next year. */
                 CURR_POR_OF_LT_DEBT_AMT, 

                /*--Subordinated debts are repayable after other debts have been paid, meaning they are more risky for the lender of the money. The debts may be secured or unsecured. Subordinated debt typically has a lower credit rating, and, therefore, a higher yield than senior debt.*/
                 CURR_POR_OF_SUB_DEBT_AMT, 

                /*--Depreciation expense is the allocated portion of the cost of a company's fixed assets that is appropriate for the accounting period indicated on the company's income statement.*/
                 DEPR_EXPNC_AMT, 

                /*-- Dividends payable are dividends that a company's board of directors has declared to be payable to its shareholders. Until such time as the company actually pays the shareholders, the cash amount of the dividend is recorded within a dividends payable account as a current liability.*/
                 DVDND_PYBL_AMT, 

                 /*--The aggregate amount of receivables to be collected from related parties where one party can exercise control or significant influence over another party, including affiliates, owners or officers and their immediate families, pension trusts, and so forth, at the financial statement date. These receivables are usually due within one year (or one business cycle).*/
                 DUE_TO_REL_PRTY_AMT, 

                 /*-- Dividends on an ESOP - a retirement plan that is designed to provide employees with an ownership interest in the company for which they work by investing primarily in stock of the employer.*/
                 ESOP_DVDND_AMT, 

                 /*--Finished goods are goods that have completed the manufacturing process but have not yet been sold or distributed to the end user.*/
                 FINSHD_GDS_INVTY_AMT, 

                /* --Assets that are purchased for long-term use and are not likely to be converted quickly into cash, such as land, buildings, and equipment.*/
                 FIX_AST_AMT, 

                 /*-- The amount of income tax that is associated with (matches) the net income reported on the company's income statement. This amount will likely be different than the income taxes actually payable, since some of the revenues and expenses reported on the tax return will be different from the amounts on the income statement.*/
                 INCM_TX_EXPNC_AMT, 

                /*-- Income tax receivable is money a company expects from tax revenue agencies. */
                 INCM_TX_RCV_AMT, 

                /*-- Interest expense is the cost of debt that has occurred during a specified period of time.*/
                 INT_EXPNC_AMT, 

                /*-- The term that companies use on their income statement for reporting the interest earned on cash temporarily held in savings accounts, certificates of deposits, or other investments. Because the interest wasn't part of the original investment, they record it separately, as interest income.*/
                 INT_INCM_AMT, 

                /*-- A long-term asset account that reports the cost of real property exclusive of the cost of any constructed assets on the property. Land usually appears as the first item under the balance sheet heading of Property, Plant and Equipment. Generally, land is not depreciated.*/
                 LAND_AMT, 

                /* -- "The expense recorded when the company does not have the rights of ownership of an asset but has been allowed the use of an asset. An operating lease is not capitalized; it is accounted for as a rental expense in what is known as ""off balance sheet financing."" */
                 LSE_RENT_EXPNC_AMT, 

                 /*-- The LIFO reserve is the difference between the accounting cost of an inventory that is calculated using the FIFO method, and one using the LIFO method. */
                 LIFO_RES_AMT, 

                 /*-- Loans and financial obligations lasting over one year. Long-term debt for a company would include any financing or leasing obligations that are to come due in a greater than 12-month period. Such obligations would include company bond issues or long-term leases that have been capitalized on a firm's balance sheet.*/
                 LT_DEBT_AMT, 

                /* --"Marketable securities are unrestricted financial instruments which can be readily sold on a stock exchange or bond exchange. Marketable securities are often classified into two groups: marketable equity securities and marketable debt securities."*/
                 MRK_SCTY_AMT,  

                 /*-- Net operating profit refers to the amount of money that a company has earned after the cost of goods sold and operating expenses have been deducted.*/
                 NET_OPER_PROF_AMT, 
                 /*-- Net sales are operating revenues earned by a company for selling its products or rendering its services. Also referred to as revenue, they are reported directly on the income statement as Sales or Net sales*/
                 NET_SALE_AMT,  
                 /*-- A non-operating asset is an asset that generates income, but is unrelated to the core operations of the company.*/
                 NON_OPER_CURR_AST_AMT, 
                 /*-- "Accumulated other comprehensive income is a subsection in equity where ""other comprehensive income"" is accumulated (summed or ""aggregated""). The balance of AOCI is presented in the Equity section of the Balance Sheet as is the Retained Earnings balance, which aggregates past and current Earnings, and past and current Dividends.".*/
                 OTH_COMP_INCM_AMT, 
                 OTH_INVTY_AMT,
                 /*-- Revenue generating assets that have not been documented elsewhere as an Operating Asset.*/
                 OTH_OPER_AST_AMT, 
                /*-- An overdraft occurs when money is withdrawn from a bank account and the available balance goes below zero. In this situation the account is said to be "overdrawn". */
                 OD_AMT,   
                 /*-- "A profitability measure that looks at a company's profits before the company has to pay corporate income tax. This measure deducts all expenses from revenue including interest expenses and operating expenses, but it leaves out the payment of tax. "*/
                 PROF_BEF_TX_AMT, 
                 /*-- "A company's stock of the basic material from which a product is manufactured."*/
                 RAW_MATL_INVTY_AMT, 
                 /*--"Reported on the income statement, it is the sum of all direct and indirect selling expenses and all general and administrative expenses of a company. "*/
                 SELL_AND_GNRL_EXPNC_AMT, 
                 /*-- Source currency code.*/
                 SRC_CRNCY_CD, 
                 /*-- The end date of the accounting period for which the financial statements were prepared.*/
                 STMT_DT,  
                /*-- Denotes the number of months in the accounting period captured by the financial statement.*/
                 STMT_MTH_CNT,  
                /*-- "A loan scheduled to be repaid in less than a year "*/
                 SHRT_TRM_LN_PYBL_AMT,  
                 /*-- Debt which ranks after other debts if a company falls into liquidation or bankruptcy.*/
                 SUB_DEBT_LBLTY_AMT, 
                /*  -- Work in Progress???.*/
                 WIP_INVTY_AMT, 
                /*-- Trade receivables (Net) are all accounts from trade, net of allowance for doutful accounts.*/
                 NET_TRD_AR_AMT,  
                /*-- The amount by which assets exceed liabilities.*/
                 TOT_NET_WORTH_AMT, 
                /*-- A balance sheet account that represents the value of all assets that are reasonably expected to be converted to cash within one year in the normal course of business.*/
                 TOT_CURR_AST_AMT, 
                 /*-- A company's debts or obligations that are due within one year. Current Liabilities appears on the balance sheet and include short term debt, accounts payable, accrued liabilities, and other debt.*/
                 TOT_CURR_LBLTY_AMT,   
                 /*-- The aggregate of all debts a company is liable for. Total Liabilities is calculated by summing all short term and long term liabilities.*/
                 TOT_LBLTY_AMT,   
                  /*-- The direct costs attributable to the production of goods sold by a company. Includes cost of materials and direct labour costs used to produce the good. It excludes indirect expenses. It appears on the income statement and is deducted from revenue to calculate gross margin. Also referred to as "cost of sales."*/
                 TOT_COGS_AMT, 
                 /*-- A companies debt, such as bonds, long-term notes payable, or debentures that will mature in more than one year or one business cycle. Classified as funded debt as it is funded by interest payments over the term of the loan. Synonymous with long term debt.*/
                 NET_FND_DEBT_AMT,  
                 /*-- Debt service coverage ratio (DSCR) also known as the debt coverage ratio (DCR) is the ratio of cash available for debt servicing for interest, principal, and lease payments. It is calculated as Net Operating Income/Debt Services. It is a popular benchmark used in the measurement of an entity's ability to produce enough cash to cover its debt (including lease payments).*/
                 DEBT_SRVC_CVRG_RATO,  
                 /*-- Investment securities that are short term, have high credit quality, and are highly liquid. They include government treasury bills, bank certificates of deposit, corporate commercial paper, and other money market instruments.*/
                 CSH_AND_EQUIV_AMT,  
                 /*-- "Identifies the accounting standard or generally accepted account principles (GAAP) applied during the preparation of the financial statements. "*/
                 ACCTG_STD_NM,  
                /*-- The sum of all raw materials, work-in-process goods, and completely finished goods that are considered to be the portion of a business's assets that are ready or will be ready for sale.*/
                 TOT_INVTY_AMT,  
                /*-- Gross Fixed Assets is the purchase price of all fixed assets (Land, buildings, equipment, machinery, vehicles, leasehold improvements) before taking into account the accumulated depreciation or consumption of the fixed assets.*/
                 GRS_FIX_AST_AMT,  
                 /*-- Net Fixed Assets is the purchase price of all fixed assets (Land, buildings, equipment, machinery, vehicles, leasehold improvements) less accumulated Depreciation.*/

                 NET_FIX_AST_AMT,  
                 /*-- A company's long term investments, where the full value will not be realized within the accounting year. Noncurrent assets are capitalized rather than expensed, meaning that the company allocates the cost of the asset over a number of years for which the asset will be in use, instead of allocating the entire cost to the accounting year in which the asset was purchased.*/
                 TOT_NON_CURR_AST_AMT,  
                 /*-- A company's long term financial obligations that are not due within the present accounting year. Examples are long-term borrowing, bonds payable, and long-term lease obligations.*/
                 TOT_NON_CURR_LBLTY_AMT, 
                 /*-- A measure of a company's financial leverage calculated by dividing its total liabilities by stockholders equity. It indicates what proportion of equity the company is using to finance its assets. Also called Debt/Equity Ratio.*/
                 TOT_LBLTY_NET_WORTH_AMT,  
                 /*-- A measure of overall sales that is not adjusted for customer discounts or returns, calculated by simply adding all sales invoices, and not including operating expenses, cost of goods sold, payment of taxes, or any other charge.*/
                 GRS_SALE_AMT,  
                 /*-- A company's revenue minus its cost of goods sold. Gross profit is a company's residual profit after selling a product/service and deducting the cost associated with its production and sale. Also called gross margin.*/
                 GRS_PROF_AMT,  
                 /*-- Those expenditures that a business incurs to engage in any activities not directly associated with the production of goods or services.*/
                 OPER_EXPNC_AMT,  
                 /*-- The financial payments and non monetary benefits provided to high level management in exchange for their work on behalf of an organization.*/
                 OFFCR_COMPEN_AMT, 
                 /*-- The sum of operating expenses which are any expenses associated with the general, sales, and administrative functions of an entity.*/
                 TOT_OPER_EXPNC_AMT,  
                 /*-- Expenses incurred related to the income earned from activities other than normal business operations, such as investment interest, foreign exchange gains (losses), and profit (loss) from the sale of non-inventory assets.*/
                 TOT_OTH_INCM_EXPNC_AMT,  
                 /*--The amount of income tax that is associated with (matches) the net income reported on the company's income statement. This amount will likely be different than the income taxes actually payable, since some of the revenue and expenses on the tax return will be different from the amounts on the income statement.*/
                 NET_INCM_TX_EXPNC_AMT,   
                 /*--This represents net profit before being adjusted for extraordinary items. Extraordinary items are gains and losses which are infrequent and atypical in nature. They are accounted for separately so they don't skew the company's regular earnings.*/
                PROF_BEF_EXTRORD_ITM_AMT,   

                /*--Often referred to as the bottom line, net profit is calculated by subtracting a company's total expenses from total revenue, thus showing what a company has earned (or lost) in a given period of time.*/
                NET_PROF_AMT,   
                /*--Comprehensive income is the change in equity of a business enterprise during a period from transactions and other events from non-owner sources. It includes all non-owner changes in equity (in contrast to net income which does not include some changes in equity).*/
                COMP_INCM_AMT,   
                /*--The capital of a business that is used in its day-to-day trading operations, calculated as the current assets minus the current liabilities.*/
                WRK_CAPTL_AMT,   
                /*--An indicator of a company's short-term liquidity. The quick ratio measures a company's ability to meet its short-term obligations with its most liquid assets. */
                QUICK_RATO,   
                 /*--A liquidity ratio that measures a company's ability to pay short-term obligations.*/
                CURR_RATO,  
                /*--A measure of the physical worth of a company, which does not include any value derived from intangible assets such as copyrights, patents and intellectual property. Tangible net worth is calculated by taking a firm's total assets and subtracting the value of all liabilities and intangible assets.*/
                TANGBL_NET_WORTH_AMT, 
                /*--Debt to tangible net worth is a derivative of the debt-to-equity ratio. */
                DEBT_TO_TOT_NET_WORTH_RATO,   
                /*--The liabilities to assets ratio is a solvency ratio that examines how much of a company's assets are financed by liabilities.*/
                LIABS_TO_AST_RATO, 
                 /*--A ratio used to determine how easily a company can pay interest on outstanding debt from its earnings before interest and taxes. */
                INT_CVRG_RATO,  
                /*--A measure of the earnings of a company that adds the interest expense, depreciation and amortization back to the net income number, but takes the tax expense into consideration.*/
                EBIDA_AMT,  
                /*--The fixed charge coverage ratio measures a firm's ability to pay all of its fixed charges or expenses with its income before interest and income taxes.*/
                FIX_CHRG_CVR_AMT,  
                 /*--An indicator of how profitable a company is relative to its total assets.*/
                RET_ON_AST_RATO,  
                /*--Provides the rate of return to the company's shareholders. Calculated as net income as a percentage of shareholders equity.*/
                RET_ON_EQTY_PCT,   
                 /*--A company's total sales revenue minus its cost of goods sold, divided by the total sales revenue, expressed as a percentage.*/
                GRS_MRGN_PCT,  
                /*--A ratio used to measure a company's pricing strategy and operating efficiency. A measurement of what proportion of a company's revenue is left over after paying for variable costs of production such as wages, raw materials, etc.*/
                OPER_PROF_MRGN_RATO,   
                /*--A ratio of profitability calculated as net income divided by revenues, or net profits divided by sales. */
                PROF_MRGN_RATO,   
                /*--The number of days that it would take for the company to collect all amounts due from customers (before deducting doubtful accounts). The point of the measurement is to determine the effectiveness of a company's credit and collection efforts in allowing credit to reputable customers, as well as its ability to collect cash from them. For example, if the credit terms are net 30 days, you would expect this to be at least 30 days.*/
                GRS_ACCT_RCV_DYS_CNT,   
                 /*--The number of days that it would take for the company to collect all amounts due from customers (net of doubtful accounts). The point of the measurement is to determine the effectiveness of a company's credit and collection efforts in allowing credit to reputable customers, as well as its ability to collect cash from them. For example, if the credit terms are net 30 days, you would expect this to be at least 30 days.*/
                NET_ACCT_RCV_DYS_CNT,  
                /*--A measure of a company's performance that gives investors an idea of how long it takes a company to turn its raw material into sales.*/
                RAW_MATL_INVTY_DYS_CNT,   
                /*--A measure of a company's performance that gives investors an idea of how long it takes a company to turn its work in progress into sales.*/
                WRK_IN_PRGRS_INVTY_DYS_CNT,   
                 /*--A measure of a company's performance that gives investors an idea of how long it takes a company to turn its finished goods into sales.*/
                FINSHD_GDS_INVTY_DYS_CNT,  
                /*--The number of days it takes a company to turn over its inventory.*/
                TOT_INVTY_DYS_CNT,   
                /*--A company's average payable period. Days payable outstanding tells how long it takes a company to pay its invoices from trade creditors, such as suppliers.*/
                ACCT_PYBL_DYS_CNT,   
                 /*--The change in total assets over completed fiscal periods.*/
                TOT_AST_GRWTH_PCT,  
                /*--The change in total liabilities over completed fiscal periods.*/
                TOT_LBLTY_GRWTH_PCT,   
                 /*--The change in net worth over completed fiscal periods.*/
                NET_WORTH_GRWTH_PCT,  
                /*--The change in operating profit over completed fiscal periods.*/
                OPER_PROF_GRWTH_PCT,   
                 /*--The change in net profit over completed fiscal periods.*/
                NET_PROF_GRWTH_PCT,  
                 /*--A lease considered a purchased asset for accounting purposes in contrast to an operating lease which is a rental.*/
                CAPTL_LSE_OBLGN_AMT,  
                /*--Estimates the ability of a company to use its cash to generate sales, eg. through purchasing inventory. Typically expressed as a ratio of working capital to sales.*/
                SALE_TO_WRK_CAPTL_RATO,   
                 /*--Ratio of sales to fixed assets measuring a company's ability to generate sales from fixed asset investments, eg. property, plant, and equipment.*/
                SALE_TO_FIX_AST_RATO,  
                 /*--The amount of physical assets a company has net of its liabilities. It represents the supposed liquidation proceeds if operations ceased immediately.*/
                EFF_TANGBL_NET_WORTH_AMT,  
                /*--Also called Debt/Equity ratio. A measure of a company's financial leverage calculated by dividing its total liabilities by shareholder's equity.*/
                DEBT_TO_WORTH_RATO,   
                /*--Also called Asset Turnover Ratio. The amount of sales or revenues generated per dollar of assets. The asset turnover ratio is a indicator of the efficiency with which a company is deploying its assets. Calculated as Asset Turnover= Sales or Revenue/Total Assets*/
                SALE_TO_TOT_AST_RATO, 
                 /*--Measures how well a company uses equity from its stockholders to generate sales. Calculated as Sales Revenue/Net Worth. Also called Sales to Equity Ratio.*/
                SALE_TO_NET_WORTH_RATO,  
                /*--The sum of current and long-term assets owned by the company.*/
                TOT_AST_AMT,   
                /*--An accounting measure calculated using a company's net earnings, before interest expenses, taxes, depreciation and amortization are subtracted, as a proxy for a company's current operating profitability, i.e., how much profit it makes with its present assets and its operations on the products it produces and sells, as well as providing a proxy for cash flow.*/
                EBITDA_AMT,   
                 /*--Consolidated financial statements are the financial statements of a group in which the assets, liabilities, equity, income, expenses and cash flows of the parent (company) and its subsidiaries are presented as those of a single economic entity, according to International Accounting Standard John - perhaps replace with more meaningful Y or N"*/
                CNSLN_IND,  
                /*--Should data flow from MRA to FACT - Y/N*/
                FACT_IND,   
                 /*--Access Group*/
                ACS_GRP_NM,  
                /*--The level of reporting i.e. Audit, Review &  Compilation, etc*/
                STMT_TYP_NM,   
                 /*--Expenses on the income statement that have not been paid or logged under accounts payable during an accounting period; in other words, obligations for goods and services provided to a company for which invoices have not yet been received. */
                ACCR_LBLTY_AMT,  
                /*--Interest payable is the interest expense that has been incurred (has already occurred) but has not been paid as of the date of the balance sheet. (The interest payable amount does not include the interest for the periods of time which follow the date of the balance sheet.)*/
                INT_PYBL_AMT,   
                /*--Ratio of EBITDA to Interest Expense.*/
                EBITDA_TO_INT_EXPNC_RATO,   
                 /*--A non-current liability that can be found on a parent company's balance sheet that represents the proportion of its subsidiaries owned by minority shareholders."*/
                MNRTY_INT_EXPNC_AMT,  
                /*--Amounts owing from related companies.*/
                AR_REL_CO_AMT,   
                F.LST_UPDT_DT_TM,
                /*--Net sales is equal to gross sales, minus discounts, customer returns, and allowances for defective/damaged merchandise. Net sales growth from one financial period to another is compared by the equation (Current period net sales- Prior period net sales)/ Prior period net sales * 100.*/
                NET_SALE_GRWTH_RATO,   
                /*--An account created in the income statement section of a business' financial statements that holds a suitable amount of funds meant to pay off upcoming interest payments. Furthermore, this type of interest is seen as an asset and unlike most conventional types of interest, it also is expensed over time.*/
                CPTLZ_INT_AMT,   
                /**********************CCRI selected Financial Module Variables listed below********************************/
                /*-- Net sales to Account Receivables: It measures how quickly the client is paid by its customers. 
                -- The lower the value the longer time it'll take for customers to pay, and longer period of exposure to higher delinquency possibility*/ 
                (Cast(net_sale_amt AS DECIMAL(25,6))/NullIfZero(net_trd_ar_amt))  AS ccri_Net_Sales_to_AR,  

                /*-- Cost of goods sold to Accounts Payables: It measures how quickly the client pays its suppliers. The lower the value
                -- the longer time the clients needs to  pay back its suppliers which indicates the worsening financial situation*/
                ( Cast(tot_cogs_amt AS DECIMAL(25,6))/NullIfZero(AP_TRD_AMT))  AS ccri_Cogs_to_AP,  

                /*-- Net funded debt to EBITDA: It measures how quickly the client pays back its debt given the stable operation of the business
                -- The higher the value, the longer period it'll take to payback the debt*/
                (Cast(net_fnd_debt_amt AS DECIMAL(25,6))/NullIfZero(ebitda_amt))  AS ccri_Net_fnd_debt_to_EBITDA , 

                /*-- Leverage ratio: It measures the capital structure of a company. The higher the value, more equity share in the capital structure
                -- the less burdened by interest payment*/																																										   
                (Cast(TOT_NET_WORTH_AMT AS DECIMAL(25,6))/NullIfZero(TOT_LBLTY_AMT))  AS ccri_leverage, 

                /*-- Debt service coverage ratio from MRA directly*/
                DEBT_SRVC_CVRG_RATO  AS ccri_DSCR,   
                /*-- Debt service coverage ratio derived*/
                Cast(EBITDA_AMT AS DECIMAL(25,6))/NullIfZero( Cast(INT_EXPNC_AMT AS DECIMAL(25,6))+Cast(CURR_POR_OF_LT_DEBT_AMT AS DECIMAL(25,6)))  AS ccri_DSCR_derived, 

                /*-- Growth of net sales*/
                NET_SALE_GRWTH_RATO AS ccri_net_sales_growth, 
                /*-- Cash and marketable securities to total assets*/
                (Cast(CSH_AMT AS DECIMAL(25,6))+Cast(MRK_SCTY_AMT AS DECIMAL(25,6)))/NullIfZero(tot_ast_amt)  AS ccri_csh_mktscty_to_ast, 
                /*--  Current ratio from MRA directly*/
                CURR_RATO AS ccri_curr_ratio, 
                /*-- Current ratio derived using the formula*/
                (Cast(TOT_CURR_AST_AMT AS DECIMAL(25,6))/NullIfZero(TOT_CURR_LBLTY_AMT)) AS ccri_curr_ratio_derived, 
                /*-- EBITDA margin*/
                (Cast(EBITDA_AMT AS DECIMAL(25,6))/NullIfZero(NET_SALE_AMT)) AS EBITDA_margin, 

                /**********************Other potential Financial Variables listed below********************************/
                /*--Net sales to Total Assets*/
                Cast(net_sale_amt AS DECIMAL(25,6))/NullIfZero(tot_ast_amt) as net_sales_to_tot_assets  ,  
                /*-- Assets to Libability*/
                Cast(tot_ast_amt AS DECIMAL(25,6))/NullIfZero(TOT_LBLTY_AMT) as assets_to_liability , 
                /*-- EBITDA to interest Expense Ratio*/
                /*EBITDA_TO_INT_EXPNC_RATO  , */
                /*-- Profit before tax to Total Assets*/
                Cast(PROF_BEF_TX_AMT AS DECIMAL(25,6))/NullIfZero(tot_ast_amt) as profit_btax_to_tot_assets 
                /*-- Total Asset amount,*/
                /*tot_ast_amt, */
                /*-- Net sales amount*/
                /*NET_SALE_AMT */
                FROM DG6V05.RSK_ANAL_STMT as F
                
                    /*Neo Added Logic */
                    INNER JOIN DG6V19.BUS_PRTFOL_CLNT AS BCP
                    ON  F.SNAP_DT  = BCP.MTH_END_DT
                    AND F.clnt_no = BCP.CLNT_NO
                    
                    INNER JOIN 
                         (SELECT CLNT_NO, SNAP_DT, max(STMT_DT) as max_STMT_DT
                           FROM DG6V05.RSK_ANAL_STMT 
                           GROUP BY CLNT_NO, SNAP_DT
                           WHERE STMT_DT <= SNAP_DT) as FF
                    ON F.CLNT_NO = FF.CLNT_NO and F.SNAP_DT = FF.SNAP_DT and FF.max_STMT_DT = F.STMT_DT

                    INNER JOIN  DDWV01.rb_clnt AS C
                    ON F.CLNT_NO = C. CLNT_NO
                            and F.SNAP_DT = C.snap_DT

                    /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                     INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                          ON  r.SNAP_DT  = c.snap_dt
                              AND r.ORG_UNT_ID = c.resp_unt
                              AND r.PARNT_HIER_TYP_ID = 80000 
                              AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)


                WHERE C.snap_DT > '{start_snap_dt}' AND C.snap_DT < '{end_snap_dt}'
                AND BCP.TOT_SN_EXP > 1000000
                and   clnt_grp NOT IN ('SB') and clnt_grp NOT IN ('OOO')
                      and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                      and c.clnt_typ=2 /* keep only non-personal clients */
                ORDER BY F.CLNT_NO, F.SNAP_DT, F.STMT_DT

            """
    return pd.read_sql(qry, connection)

In [47]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_financials(start_snap_dt, end_snap_dt)
df1.shape

(129983, 138)

In [48]:
df1.to_csv(raw_data_dir+'financials_y1.csv', index=False)

In [49]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_financials(start_snap_dt, end_snap_dt)
df2.shape

(322089, 138)

In [50]:
df2.to_csv(raw_data_dir+'financials_y2.csv', index=False)

In [51]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_financials(start_snap_dt, end_snap_dt)
df3.shape

(337719, 138)

In [52]:
df3.to_csv(raw_data_dir+'financials_y3.csv', index=False)

In [53]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_financials(start_snap_dt, end_snap_dt)
df4.shape

(550653, 138)

In [54]:
df4.to_csv(raw_data_dir+'financials_y4.csv', index=False)

# Fac_general

In [55]:
def pull_fac_general(start_snap_dt, end_snap_dt):    
    qry = f"""\
            select 
            /* month end date */
                   C.SNAP_DT as mth_end_dt, 
            /* client number */
                   C.clnt_no , 
            /* single name ID */
                     CASE
                           WHEN SN.SN_ID IS NULL THEN C.CLNT_NO
                           ELSE SN.SN_ID
                         END AS SN_ID,
              
            /* client group, used to define commercial and small business clients */
            case
                        when (c.BSC in (190,192,200,202,250,490,590)
                                or (c.BSC in (310,500,502,504,505,510) and 
                                    (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203,
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL))
                             ) then 'COMM'
                
                        when (c.BSC in (550,554,555)
                                and (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203,
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL)
                             ) then 'AG'
                        when c.BSC in (330) then 'PUB'
                        when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                and c.CURR_CMI_NO in (100,101,102,103,
                                                        200,201,202,203,
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503)
                             )then 'SB'
                        else 'OOO'
                    end as Clnt_Grp,
                   /* arrangement id/facility id*/ 
                   F.ar_id as fac_id,     
                   /*facility type*/ 
                   F.fac_tp_1 as fac_type , 
                   /*authorized credit status*/ 
                   F.auth_sts as fac_sts ,  
                   /*facility lied code*/ 
                   F.fac_lied_cd,   
                   /*instrument group code*/ 
                   F.inst_grp_cd, 
                   /*facility authorization amount*/
                   F.fac_auth_amt, 
                   /*facility outstnding amount*/
                   F.fac_os_amt 
            
            FROM DDWV01.rb_clnt AS C
            
            /*Neo Added Logic */
                     INNER JOIN DG6V19.BUS_PRTFOL_CLNT AS BCP
                        ON  C.SNAP_DT  = BCP.MTH_END_DT
                        AND c.clnt_no = BCP.CLNT_NO  
            
            
                    /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                 INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                      ON  r.SNAP_DT  = c.snap_dt
                          AND r.ORG_UNT_ID = c.resp_unt
                          AND r.PARNT_HIER_TYP_ID = 80000 
                          AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)
            
                /* Used to get client SN ID */
                LEFT JOIN DG6V19.CLNT_SN_HIST as SN
                             ON C.CLNT_NO = SN.CLNT_NO
                            AND C.snap_DT = SN.MTH_END_DT
                              
                /* Facility info */
                    INNER JOIN DG6V19.BUS_PRTFOL_FAC as F
                        ON  C.clnt_no = F.clnt_no
                            and C.snap_dt = F.mth_end_dt
            where 
                  clnt_grp NOT IN ('OOO', 'SB')
                  AND TOT_SN_EXP > 1000000
                  and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                  and c.clnt_typ=2 /* keep only non-personal clients */
                  and c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'
        """
    return pd.read_sql(qry, connection)

In [56]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_fac_general(start_snap_dt, end_snap_dt)
df1.shape

(1670352, 11)

In [57]:
df1.to_csv(raw_data_dir+'fac_general_y1.csv', index=False)

In [58]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_fac_general(start_snap_dt, end_snap_dt)
df2.shape

(1764193, 11)

In [59]:
df2.to_csv(raw_data_dir+'fac_general_y2.csv', index=False)

In [60]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_fac_general(start_snap_dt, end_snap_dt)
df3.shape

(1868521, 11)

In [61]:
df3.to_csv(raw_data_dir+'fac_general_y3.csv', index=False)

In [62]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_fac_general(start_snap_dt, end_snap_dt)
df4.shape

(3020126, 11)

In [63]:
df4.to_csv(raw_data_dir+'fac_general_y4.csv', index=False)

# BDA

In [64]:
def pull_bda_general(start_snap_dt, end_snap_dt):    
    qry = f"""\
               select 
                /* month end date */
                       C.SNAP_DT as mth_end_dt, 
                /* client number */
                       C.clnt_no , 
                /*arrangement id*/
                      A.ar_id,             
                      'A' || trim(A.ar_id) as t_ar_id, /* Added by QH on 2018-11-07 */
                /* client group, used to define commercial and small business clients */
                case
                            when (c.BSC in (190,192,200,202,250,490,590)
                                    or (c.BSC in (310,500,502,504,505,510) and 
                                        (c.CURR_CMI_NO not in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503) or c.CURR_CMI_NO is NULL))
                                 ) then 'COMM'
                    
                            when (c.BSC in (550,554,555)
                                    and (c.CURR_CMI_NO not in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503) or c.CURR_CMI_NO is NULL)
                                 ) then 'AG'
                            when c.BSC in (330) then 'PUB'
                            when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                    and c.CURR_CMI_NO in (100,101,102,103,
                                                            200,201,202,203,
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503)
                                 )then 'SB'
                            else 'OOO'
                        end as Clnt_Grp,
                /*currency code*/
                    O.crncy_cd,    
                /*open/close status*/
                    O.op_cls_sts,  
                /*balance*/
                    DD.bal,    
                /*balance-backdated*/ 
                    DD.bal_bkdt,  
                /* MIF service ID*/
                    D.mif_srvc_id, 
                /* deposit amount*/
                    D.dep_amt, 
                /*deposits count*/
                    D.deps_cnt, 
                /* minimum cycle balance*/
                    D.cyc_minm_bal, 
                /* transfer credit amount*/
                    D.tsfr_cr_amt, 
                /* transfer credit count*/
                    D.tsfr_cr_cnt, 
                /* transfer debit amount*/
                    D.tsfr_dr_amt, 
                /* transfer debit count*/
                    D.tsfr_dr_cnt, 
                /* average credit balance*/
                    D.avg_cr_bal, 
                /* average credit balance backdate*/
                    D.avg_cr_bal_bkdt, 
                /* average debit balance */
                    D.avg_dr_bal, 
                /* average debit balance backdate*/
                    D.avg_dr_bal_bkdt, 
                /* overdraft count*/
                    D.od_cnt, 
                /* overdraft accumulated balance*/
                    D.od_accum_bal, 
                /* overdraft current number of days*/
                    D.od_days_in_mth,
                /* overdraft interest charged*/ 
                    D.od_int_chrg, 
                /* overdraft  current number of days*/
                    D.od_curr_no_of_days, 
                /* other credit amount*/ 
                    D.oth_cr_amt, 
                /* other credit count*/
                    D.oth_cr_cnt, 
                /* other debit amount*/
                    D.oth_dr_amt, 
                /* other debit count*/
                    D.oth_dr_cnt, 
                /* not sufficient fund charge from previous month end*/
                    D.nsf_chrg_mtd,
                /* not sufficient fund times count*/ 
                    D.nsf_times_cnt, 
                /* COBC  indicator*/
                    D3.cobc_cbr_type, 
                /* Overdraft limit*/ 
                    D3.spcl_od_lmt,
                /* overdraft limit type*/ 
                    D3.od_lmt_type,
                /* excess item charge*/
                    D4.excs_itm_chrg, 
                /* excess days in month count including last business day  */
                    D4.excess_crrnt_mth_cnt_incl,
                /* excess days in month count including last business day */ 
                    D4.excess_crrnt_mth_cnt_excl, 
                /* minimum balance during month*/
                    B.min_bal,
                /* maximum balance during month*/ 
                      B.max_bal, 
                /* average balance during month*/
                      B.avg_bal          
                
                FROM DDWV01.rb_clnt AS C
                
                /*Neo Added Logic */
                     INNER JOIN DG6V19.BUS_PRTFOL_CLNT AS BCP
                        ON  C.SNAP_DT  = BCP.MTH_END_DT
                        AND c.clnt_no = BCP.CLNT_NO                     
                    
                        /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                     INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                          ON  r.SNAP_DT  = c.snap_dt
                              AND r.ORG_UNT_ID = c.resp_unt
                              AND r.PARNT_HIER_TYP_ID = 80000 
                              AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)
                    
                    /* Inner join with arrangement ownership history to pull primary clients on BDA accounts */
                     INNER JOIN DG6V19.arngmnt_own_hist as A
                         on A.clnt_no = C.clnt_no
                            and A.mth_end_dt = C.snap_dt
                            and A.prmry_alt_ind = 'P'
                            and A.mif_srvc_id = 2
                
                    /* Arrangement history table */
                     INNER JOIN DG6V19.arngmnt_hist as O
                       ON O.mth_end_dt = A.mth_end_dt
                          and O.mif_srvc_id = 2
                          and O.mif_srvc_id = A.mif_srvc_id /*-- add on 2018-07-03, checked by Kevin*/
                          and O.ar_id = A.ar_id
                
                    /* Deposit account monthly stats history table */
                     INNER JOIN DG6V19.dep_acct_mth_stt_hist as D
                         on D.mif_acct_no = A.mif_acct_no 
                            and D.mif_srvc_id = A.mif_srvc_id 
                            and D.mth_end_dt = A.mth_end_dt 
                            /*--and D.mth_end_dt LE '2014-10-31'*/
                            and D.mif_srvc_id = 2 
                
                    /* Account history table */
                     INNER JOIN DG6V19.acct_hist as DD
                         on DD.mif_acct_no = A.mif_acct_no 
                            and DD.mif_srvc_id = A.mif_srvc_id 
                            and DD.mth_end_dt = A.mth_end_dt 
                            /*--and DD.mth_end_dt LE '2014-10-31'*/
                            and DD.mif_srvc_id = 2 
                
                    /* Deposit account history table */
                     INNER JOIN DG6V19.dep_acct_hist as D3
                         on D3.mif_acct_no = A.mif_acct_no 
                            and D3.mif_srvc_id = A.mif_srvc_id
                            and D3.mth_end_dt = A.mth_end_dt 
                            /*--and D3.mth_end_dt LE '2014-10-31'*/
                            and D3.mif_srvc_id = 2 
                
                    /* Deposit acct mthly table */
                     INNER JOIN DDWV01.depst_acct_mthly as D4
                         on D4.month_end_date = A.mth_end_dt
                            and D4.dw_srvc_id = A.mif_srvc_id
                            and D4.ar_id = A.ar_id
                            /*--and D4.month_end_date LE '2014-10-31'*/
                            and D4.dw_srvc_id = 2
                
                    /* min/max/avg balance table */
                     LEFT JOIN 
                     (SELECT 
                         VT_CLNT_CORE.mth_end_dt,
                         VT_CLNT_CORE.ar_id,
                         MIN(
                           case
                               when AR_BAL_DLY.snap_dt GE '2012-11-23' then zeroifnull(AR_BAL_DLY.bal_aft_bkdt)
                                else zeroifnull(AR_BAL_DLY.bal)
                           end
                           ) as min_bal,
                        MAX(
                           case
                             when AR_BAL_DLY.snap_dt GE '2012-11-23' then zeroifnull(AR_BAL_DLY.bal_aft_bkdt)
                             else zeroifnull(AR_BAL_DLY.bal)
                           end
                           ) as max_bal,
                         
                        AVG(
                           case
                             when AR_BAL_DLY.snap_dt GE '2012-11-23' then zeroifnull(AR_BAL_DLY.bal_aft_bkdt)
                             else zeroifnull(AR_BAL_DLY.bal)
                           end
                           ) as avg_bal
                
                      FROM 
                         (SELECT 
                             CL.snap_dt as mth_end_dt, 
                             AR.ar_id, 
                             case
                            when (CL.BSC in (190,192,200,202,250,490,590)
                                    or (CL.BSC in (310,500,502,504,505,510) and 
                                        (CL.CURR_CMI_NO not in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503) or CL.CURR_CMI_NO is NULL))
                                 ) then 'COMM'
                    
                            when (CL.BSC in (550,554,555)
                                    and (CL.CURR_CMI_NO not in (100,101,102,103,
                                                                200,201,202,203,
                                                                300,301,302,303,
                                                                401,402,403,
                                                                501,502,503) or CL.CURR_CMI_NO is NULL)
                                 ) then 'AG'
                            when CL.BSC in (330) then 'PUB'
                            when (CL.BSC in (310,500,502,504,505,510, 550,554,555) 
                                    and CL.CURR_CMI_NO in (100,101,102,103,
                                                            200,201,202,203,
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503)
                                 )then 'SB'
                            else 'OOO'
                        end as Clnt_Grp  
                
                          FROM
                            DDWV01.rb_clnt  as CL
                
                            INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK as RO
                             ON  RO.SNAP_DT  = CL.snap_DT
                                  and RO.ORG_UNT_ID = CL.resp_unt
                                    and RO.PARNT_HIER_TYP_ID = 80000 
                                    and RO.ROLL_ID_RANK_600 in (80013,80029,80146,82314,83080,83081,83082,83091)
                            
                            INNER JOIN DG6V19.arngmnt_own_hist as AR
                             ON AR.clnt_no = CL.clnt_no
                                 and AR.mth_end_dt = CL.snap_dt
                                 and AR.prmry_alt_ind = 'P'
                                 and AR.mif_srvc_id = 2
                
                         WHERE 
                           clnt_grp NOT IN ('OOO','SB')
                           and CL.snap_dt > '{start_snap_dt}' AND CL.snap_dt < '{end_snap_dt}' /*modified by Qianyi on 2018-11-07*/
                         ) AS VT_CLNT_CORE,   
                        
                        DDWV01.ar_bal_dly AS AR_BAL_DLY
                             
                      WHERE EXTRACT (YEAR FROM VT_CLNT_CORE.mth_end_dt) = EXTRACT (YEAR FROM AR_BAL_DLY.snap_dt )
                           and EXTRACT (MONTH FROM VT_CLNT_CORE.mth_end_dt) = EXTRACT (MONTH FROM AR_BAL_DLY.snap_dt )
                           and VT_CLNT_CORE.ar_id = AR_BAL_DLY.ar_id
                      GROUP BY 1,2) as B
                         
                    ON B.mth_end_dt = A.mth_end_dt
                       and B.ar_id = A.ar_id

                
                where 
                      clnt_grp NOT IN ('OOO','SB')
                      AND TOT_SN_EXP > 1000000
                      and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                      and c.clnt_typ=2 /* keep only non-personal clients */
                      and c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'
        """
    return pd.read_sql(qry, connection)

In [65]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_bda_general(start_snap_dt, end_snap_dt)
df1.shape

(798582, 41)

In [66]:
df1.to_csv(raw_data_dir+'bda_general_y1.csv', index=False)

In [67]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_bda_general(start_snap_dt, end_snap_dt)
df2.shape

(832527, 41)

In [68]:
df2.to_csv(raw_data_dir+'bda_general_y2.csv', index=False)

In [69]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_bda_general(start_snap_dt, end_snap_dt)
df3.shape

(867904, 41)

In [70]:
df3.to_csv(raw_data_dir+'bda_general_y3.csv', index=False)

In [71]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_bda_general(start_snap_dt, end_snap_dt)
df4.shape

(1377650, 41)

In [72]:
df4.to_csv(raw_data_dir+'bda_general_y4.csv', index=False)

# Prod_Char

In [73]:
def pull_prod_char(start_snap_dt, end_snap_dt):    
    qry = f"""\
            select 	
                    A.clnt_no,
                    A.capture_dt,
                    /*-- DDA behavioral Information*/
                    dda_any_clct_typ,  /*-- dda any collection type*/
                    dda_wrst_rstrnt_typ, /* -- dda worst restraint type*/
                    dda_probe_data_sole_mths_cnt, /*-- dda probe data sole account months count*/
                    dda_probe_data_joint_mths_cnt,  /*-- dda probe data joint account months count*/
                    dda_probe_data_tot_mths_cnt, /*-- dda probe data total months count*/
                    dda_days_since_lst_cg_dep, /*-- dda days since last customer generated deposit*/
                    dda_curr_to_avg_pay_dep_ratio,  /*-- dda current to average payroll deposit ratio*/
                    dda_mths_1_3_avg_cg_dep, /* -- dda months 1-3 average customer generated deposit*/
                    dda_mths_1_12_avg_cg_dep,  /*-- dda months 1-12 average customer generated deposit*/
                    dda_mths_1_6_sole_expct_val_dp,  /*-- dda months 1-6 sole owner expected value deposits*/
                    dda_mths_1_6_joint_expct_valdp, /*-- dda months 1-6 joint owner expected value deposit*/
                    dda_mths_1_6_tot_expct_val_dep,  /*-- dda months 1-6 total expected value deposits*/
                    dda_curr_dep_to_expct_dep_rati, /*-- dda current deposit to expected deposit ratio*/
                    dda_curr_qtr_dep_to_prev_dep_r,  /*		-- dda current quarter deposit to previous deposit ratio*/
                    dda_prev_qtr_dep_to_dep_mth7_9, /*		-- dda previous quarter deposit to deposit mth7-9 ratio*/
                    dda_ann_comp_of_qtr_dep_ratio,  /*		--dda annual comp of quarter deposit ratio*/
                    dda_mths_1_12_regity_of_deps, /*		-- dda months 1-12 regularity of deposits*/
                    dda_tot_od_lmt,  /*		-- dda total overdraft limit*/
                    dda_mths_1_3_avg_emth_bal,  /*		-- dda months 1-3 average end of month balance*/
                    dda_tot_courtesy_od_lmt, /*		-- dda total courtesy overdraft limit*/
                    dda_mths_1_3_sole_avg_dbs,  /*		-- dda months 1-3 sole owner average debits*/
                    dda_mths_1_3_joint_avg_dbs,  /*		-- dda months 1-3 joint owner average debits*/
                    dda_mths_1_3_tot_avg_dbs, /*		--dda months 1-3 total of average debit*/
                    dda_curr_mth_wrst_dys_in_debit,  /*		-- dda current month worst days in debit*/
                    dda_mths_1_3_wrst_dys_in_debit, /*		-- dda months 1-3 worst days in debit*/
                    dda_mths_4_6_wrst_dys_in_debit,   /*		-- dda months 4-6 worst days in debit*/
                    dda_curr_mth_tot_days_in_exces,   /*		-- dda current month total days in excess*/
                    dda_mths_1_3_tot_days_in_exces,   /*		-- dda months 1-3 total days in excess*/
                    dda_mths_4_6_tot_days_in_exces,  /*		-- dda months 4-6 total days in excess*/
                    dda_curr_mth_tot_no_of_rets,  /*		-- dda current month total number of returns*/
                    dda_mths_1_6_tot_no_of_rets,  /*		--dda months 1-6 total number of returns*/
                    dda_tot_no_of_sole_own_open_ac, /*		--dda total number of sole owner open account*/
                    dda_tot_no_of_joint_open_acct,  /*		-- dda total number of joint open account*/
                    dda_curr_mth_drs_cnt,  /*		-- dda current month debits count*/
                    mn_dda_mths_1_12_hi_low_bal_sw,  /*		-- main dda months 1-12 high/low balance swing*/
                    mn_dda_frcst_bal_to_avg_dep_rt,  /*		-- main dda forecast balance to average deposit ratio*/
                    mn_dda_frcst_qtr_bal_to_avg_dp, /*		--main dda forecast quarter balance to average deposit ratio*/
                    mn_dda_open_to_buy_ratio, /*		-- main dda open to buy ratio*/
                    mn_dda_qtr_chg_in_frcst_bal_rt, /*		-- main dda quarter change in forecast balance ratio*/
                    mn_dda_mthly_hi_bal, /*		-- main dda monthly high balance*/
                    dda_val_sole_own_pay_crs,  /*		-- dda value sole owner payroll credits*/
                    dda_val_joint_own_pay_crs,  /*		-- dda value joint owner payroll credits*/
                    dda_val_tot_own_pay_crs, /*		-- dda value total payroll credits*/
                    dda_val_deps_sole_net_pay_crs,  /*		-- dda value deposits sole owner net payroll credit*/
                    dda_val_deps_joint_net_pay_crs, /*		-- dda value deposits joint owner net payroll credits*/
                    dda_val_deps_tot_net_pay_crs,  /*		-- dda value deposits total net payroll credits*/
                    dda_av_pay_crdt_last12_s, /*		-- average value of payroll credits last 12 months (sole)*/
                    dda_av_pay_crdt_last12_j,  /*		--average value of payroll credits last 12 months (joint)*/
                    dda_dys_snc_excs_cnt, /*		-- days since excess*/
                    invst_tot_sole_opentrm_dp_cnt,  /*		--investment total sole owner open term deposit count*/
                    invst_tot_joint_opentrm_dp_cnt,  /*		-- investment total joint open term deposit count*/
                    ln_wrst_curr_drs_scor,  /*		-- loan worst current drs score*/
                    ln_wrst_ln_sts, /*		-- loan worst loan status*/
                    ln_wrst_curr_days,  /*		-- loan worst current days delinquent*/
                    ln_tot_os_arrears_amt, /*		-- loan total outstanding arrears amount*/
                    ln_mths_1_3_wrst_days_dlq, /*		-- loan months 1-3 worst days delinquent*/
                    ln_mths_4_6_wrst_days_dlq, /*		-- loan months 4-6 worst days delinquent*/
                    ln_mths_7_9_wrst_days_dlq,  /*		-- loan months 7-9 worst days delinquent*/
                    ln_mths_10_12_wrst_days_dlq, /*		-- loan months 10-12 worst days delinquent*/
                    ln_mths_1_6_30days_dlq_cnt,  /*		-- loan months 1-6 30 days delinquent count*/
                    ln_mths_snc_lst_60plus_dys_dlq,  /*		-- loan months since last 60 plus days delinquent*/
                    ln_non_revl_tot_emth_bal, /*		-- loan non-revolving total end of month balance*/
                    ln_non_revl_tot_minm_pymt,  /*		-- loan non-revolving total minimum payment*/
                    ln_non_revl_pymt_typ,  /*		-- loan non-revolving payment type*/
                    ln_non_revl_any_ln_ins_typ, /*		-- loan non-revolving any loan insurance type*/
                    ln_non_revl_max_mths_to_run, /*		-- loan non-revolving maximum months to run*/
                    ln_non_revl_mths_since_most_re, /*		-- loan non-revolving months since most recent open loan*/
                    ln_non_revl_most_recnt_open_ln,  /*		-- loan non-revolving most recent open loan purpose code*/
                    ln_non_revl_tot_sole_open_cnt, /*		-- loan non-revolving total sole owner open loan count*/
                    ln_non_revl_tot_joint_open_cnt,  /*		-- loan non-revolving total joint open loan count*/
                    ln_non_revl_12_mth_wrst_daydlq, /*		-- worst days delinquent for non-revolving loans in the last 12 months*/
                    ln_non_revl_wrst_curr_dys_dlq,  /*		-- worst current days delinquent for loans*/
                    ln_tot_cola_val,  /*		-- loan total collateral value*/
                    ln_revl_mth_1_3_emth_avg_bal, /*		-- loan revolving month 1-3 end of month average balance*/
                    ln_revl_tot_minm_pymt,  /*		--loan revolving total minimum payment*/
                    ln_revl_tot_lmt,  /*		-- loan revolving total limit*/
                    ln_revl_mths_1_3_avg_lmt_utlz, /*		-- loan revolving month 1-3 average limit utilization*/
                    ln_revl_utlzn_ratio_to_prev_qt,  /*		-- loan revolving utilization ratio to previous quarter*/
                    ln_revl_avg_pymt_stmt_bal_rat, /*		-- loan revolving average payment to statement balance ratio*/
                    ln_revl_chg_avg_pymt_stmt_rat,/*		-- loan revolving change in average payment to statement balance ratio*/
                    ln_revl_sole_own_tot_open_cnt,/*		-- loan revolving sole owner total open count*/
                    ln_revl_joint_own_tot_open_cnt,  /*		-- loan revolving joint owner total open count*/
                    ln_hi_int_rt,  /*		-- loan highest interest rate*/
                    ln_most_recntly_open_sub_purps, /*		-- loan most recently opened sub purpose code*/
                    mortg_wrst_curr_days_dlq,  /*		-- mortgage worst current days delinquent*/
                    mortg_mths_1_3_wrst_days_dlq,  /*		-- mortgage months 1-3 worst days delinquent*/
                    mortg_mths_4_6_wrst_days_dlq, /*		-- mortgage months 4-6 worst days delinquent*/
                    mortg_mths_7_9_wrst_days_dlq,  /*		-- mortgage months 7-9 worst days delinquent*/
                    mortg_mths_10_12_wrst_days_dlq, /*		-- mortgage months 10-12 worst days delinquent*/
                    mortg_mths_1_6_30days_dlq_cnt, /*		-- mortgage months 1-6 30 days delinquent count*/
                    mortg_mths_since_lst_60_plus_d, /*		-- mortgage months since last 60 plus days delinquent*/
                    mortg_mths_since_most_recnt_op,  /*		-- mortgage months since most recent open*/
                    mortg_scty_aprsd_val_less_os, /*		-- mortgage security total appraised value less total outstanding*/
                    mortg_emth_tot_bal,  /*		-- mortgage end of month total balance*/
                    mortg_tot_mthly_repay,  /*		-- mortgage total monthly repayment*/
                    mortg_ratio_osbal_ovr_aprsd_vl, /*		-- mortgage ratio outstanding balance over appraised value*/
                    mortg_tot_sole_own_open_cnt,  /*		-- mortgage total sole owner open count*/
                    mortg_tot_joint_own_open_cnt,  /*		-- mortgage total joint owner open count*/
                    mortg_lst_12_mth_wrst_dys_dlq,  /*		-- worst days delinquent for mortgage in the last 12 months*/
                    visa_age_of_oldest_acct, /*		-- visa age of oldest account*/
                    visa_mths_1_3_avg_stmt_bal,  /*		-- visa months 1-3 average statement balance*/
                    visa_mths_1_3_avg_drs,  /*		-- visa months 1-3 average debits*/
                    visa_bhvr_scor, /*		-- visa behavioral score*/
                    visa_ratio_mths_1_3_avg_pymt_o,  /*		-- visa ratio months 1-3 average payment over expected payment*/
                    visa_wrst_curr_sts_reas_cd,  /*		-- visa worst current status / reason code*/
                    visa_curr_wrst_days_dlq, /*		-- visa current worst days delinquent*/
                    visa_mths_1_3_wrst_days_dlq,  /*		-- visa months 1-3 worst days delinquent*/
                    visa_mths_4_6_wrst_days_dlq,  /*		-- visa months 4-6 worst days delinquent*/
                    visa_tot_no_open,  /*		-- visa total number open accounts count*/
                    visa_bnkruptcy_scor, /*		-- visa bankruptcy score*/
                    visa_bnk_init_lmt_decr,  /*		-- bank initiated visa decrease*/
                    visa_lst_12_mth_wrst_dys_dlq, /*		-- worst days delinquent for visa in the last 12 months*/
                    visa_mth_snc_bnk_price_incr,  /*		-- months since last bank initiated visa price increase*/
                    rcl_campgn_id,  /*		-- rcl campaign id*/
                    rcl_lst_12mth_wrst_dys_dlq_cnt,  /*		-- worst days delinquent rcl last 12 months*/
                    rcl_mth_since_lst_prc_incr, /*		-- months since last bank initiated rcl price increase*/
                    rcl_curr_wrst_dys_dlq_cnt,  /*		-- worst current days delinquent for rcl*/
                    rcl_tot_scty_lmt_amt,   /*		-- total rcl secured limits*/
                    rcl_tot_unscty_lmt_amt,  /*		-- total rcl unsecured limits*/
                    a.probe_sys_src_id,/*		-- probe system source identifier*/
                    a.dly_or_mth_typ_cd, /*		-- daily or monthly type code*/
                    a.grp_key,  /*		-- group key*/
                    avg_cust_dep_svng_l6m_tot_bal,  /*		-- average customer deposit savings l6m total balance*/
                    tot_avg_dr_trnvr_l12m_amt, /*		-- total average debit turnover l12m amount*/
                    tot_avg_cr_trnvr_l12m_amt,  /*		-- total average credit turnover l12m amount*/
                    tot_avgdr_trnvr_totavg_cr_l12m,  /*		-- total avg debit turnover to total avg credit turnover l12m amount*/
                    tot_expct_cr_trnvr_amt, /*		-- total expected credit turnover amount*/
                    tot_expct_cr_trnvr_tot_unscty,  /*		-- total expected credit turnover to total unsecured lending*/
                    tot_expct_cr_trnvr_tot_secur, /*		-- total expected credit turnover to total secured lending*/
                    tot_no_of_pymt_rvrs_cnt,  /*		-- total number of payment reversal count*/
                    tot_no_of_pymt_rvrs_l3m_cnt,  /*		-- total number of payment reversal l3m count*/
                    tot_no_of_pymt_rvrs_l4_6m_cnt, /*		-- total number of payment reversal l4-6m count*/
                    no_of_pymt_rvrs_l3m_l4_6m_pct,  /*		--number of payment reversal l3m to l4-6m percent*/
                    mth_snc_lst_pymt_rvrs_cnt
                    
                    /*-- More features added from CLNT_PROD_CHARSTC_VRB table*/
                    CHQ_LMT_UTLZN_AVG_LDGR_BAL_CM,
                    CHQ_LMT_UTLZN_AVG_LDGR_BAL_L6M,
                    CHQ_LMT_UTLZN_FOR_MIN_BAL_CM,
                    CHQ_LMT_UTLZN_FOR_MIN_L6M_BAL,
                    CHQ_MAX_DYS_IN_CONT_EXCS_CNT,
                    CHQ_MAX_NO_OF_DYS_IN_DR_CNT,
                    CHQ_MAX_NO_DYS_IN_DR_L3M_CNT,
                    CHQ_MAX_DYS_IN_DR_L3M_L4_6MPCT,
                    CHQ_MAX_NO_DYS_IN_DR_L4_L6M,
                    CHQ_MINM_BAL_L3M_AMT,
                    MTH_SNC_LST_CHQ_CA_ACCT_OPEN,
                   CHQ_TOT_AVG_CR_TRNVR_L3M_AMT,
                   CHQ_TOT_DYS_IN_EXCS_CNT,
                   CHQ_TOT_DYS_IN_EXCS_L3M_CNT,
                   CHQ_TOT_DYS_IN_EXCS_L4_L6M_CNT,
                   CU_NO_OF_CHQ_CA_ACCT_CNT,
                  CU_NO_INDRCT_LEND_LN_LIVE_CNT,
                  CU_NO_OF_MC_LIVE_CNT,
                  CU_NO_OF_RCL_ACCT_LIVE_CNT,
                  RCL_NO_REVL_CR_ACCT_LIVE_CNT,
                 CU_NO_OF_SECUR_HMLN_RCL_LIVE,
                 LN_FT_NO_OF_HOM_LN_LIVE_CNT,
            CU_NO_OF_SECUR_RCL_LIVE_CNT,
            CU_NO_OF_SECUR_TRM_LN_LIVE_CNT,
            CU_NO_OF_TD_CA_ACCT_LIVE_CNT,
            CU_NO_OF_TRM_DEP_LIVE_CNT,
            HEF_FACILITY_BALANCE,
            HEF_FACILITY_LIMIT,
            HEF_FACILITY_REDUCING_IND,
            HEF_FACILITY_WORST_EVER_DLQ,
            LN_FT_NO_OF_PERS_LN_LIVE_CNT,
            LN_NUM_GOV_GUAR_LNS_CNT,
            LN_NUM_INDIRECT_LNS_CNT,
            LN_NO_OF_FIX_TRM_ACCT_LIVE_CNT,
            MINM_MTH_TO_RUN_ON_PERS_LN_CNT,
            MTH_SNC_LST_CR_ACCT_OPEN_CNT,
            MTH_SINCE_LST_LMT_CHG_CNT,
            MTH_SNC_LST_PERS_LN_OPEN_CNT,
            NON_CHQ_FT_MAX_DLQ_CNT,
            NON_CHQ_FT_MAX_L6M_DLQ_CNT,
            NON_CHQ_FT_MAX_L7_12M_DLQ_CNT,
            NON_CHQ_FT_MTH_SNC_HL_ACCT_OPN,
            NON_CHQ_FT_TOTEXPSR_UNSCTY,
            NON_CHQ_FT_TOTEXPSR_UNSCTY_FCT,
            NON_CHQ_FT_WRST_EXT_BHVR_SCOR,
            NON_CHQ_MTH_MST_RCNT_2CYL_DLQ,
            NON_CHQ_RCL_AVG_PYMT_L3M_BAL,
            NON_CHQ_RCL_AVG_PYMT_L3M_L4_6M,
            NON_CHQ_RCL_AVG_PYMT_L4_6M_BAL,
            NON_CHQ_RCL_CUST_LMT_L6M_UTLZN,
            NON_CHQ_RCL_CUST_LMT_UTLZN_AMT,
            NON_CHQ_RCL_MAX_DLQ_CNT,
            NON_CHQ_RCL_MAX_L6M_DLQ_CNT,
            NON_CHQ_RCL_MAX_L7_12M_DLQ_CNT,
            NON_CHQ_RCL_TOTEXPSR_UNSCTY,
            NON_CHQ_RCL_TOTEXPR_UNSCTY_FCT,
            NON_CHQ_RCL_TOTVAL_L3M_CSH_DBT,
            NON_CHQ_RCL_TOTVAL_L3M_CSH_ADV,
            RCL_NO_OF_CR_CRD_LIVE_CNT,
            TOT_AVG_DR_TRNVR_L3M_AMT,
            TOT_DEP_SVNG_BAL_AMT,
            TOT_EXPSR_CHQ_UNSCTY_AMT,
            TOT_NET_TRNVR_AMT
                    
            from DDWV11.clnt_bhvr_charstc AS A
            
            INNER JOIN (Select clnt_no, 
                        MTH_END_DT, 
                        TOT_SN_EXP 
                        FROM DG6V19.BUS_PRTFOL_CLNT WHERE TOT_SN_EXP >1000000) C
            ON A.clnt_no = C.clnt_no and A.capture_dt = C.MTH_END_DT
            
            
            INNER JOIN DDWV11.clnt_prod_charstc_vrb as B
            ON A.clnt_no = B.clnt_no
                  and A.capture_dt = B.capture_dt
            where A.pers_bus_flg_typ = 'B' and A.capture_dt > '{start_snap_dt}' AND A.capture_dt < '{end_snap_dt}'
    """
    return pd.read_sql(qry, connection)

In [74]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_prod_char(start_snap_dt, end_snap_dt)
df1.shape

(460593, 198)

In [75]:
df1.to_csv(raw_data_dir+'prod_char_y1.csv', index=False)

In [76]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_prod_char(start_snap_dt, end_snap_dt)
df2.shape

(498065, 198)

In [77]:
df2.to_csv(raw_data_dir+'prod_char_y2.csv', index=False)

In [78]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_prod_char(start_snap_dt, end_snap_dt)
df3.shape

(530605, 198)

In [79]:
df3.to_csv(raw_data_dir+'prod_char_y3.csv', index=False)

In [80]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_prod_char(start_snap_dt, end_snap_dt)
df4.shape

(863901, 198)

In [81]:
df4.to_csv(raw_data_dir+'prod_char_y4.csv', index=False)

# Bhvr_Char

In [82]:
def pull_bhvr_char(start_snap_dt, end_snap_dt):    
    qry = f"""\
                select  
                /*client number*/
                    A.clnt_no, 
                /*capture date*/
                    A.capture_dt, 
                /*customer type*/
                    A.cust_typ, 
                /*personal business flag type */
                    A.pers_bus_flg_typ, 
                /*consumer business flag type*/
                    A.cnsmr_bus_flg_typ, 
                /*triage status*/
                    A.triage_sts,
                /*Months since last triage event */
                    A.tm_mths_since_lst_triage_evnt, 
                /*last triage event identifier*/
                    A.lst_triage_evnt_id, 
                /*business current credit code*/
                    A.bus_curr_cr_cd,  
                /*gbix node identifier*/
                    A.gbix_node_id, 
                /*last customer segmentation code */
                    A.lst_cust_segmt_cd,  
                    
                    
                /******************************
                Strength of Business Income
                ********************************/
                
                /*client estimated gross income*/
                    A.est_grs_incm, 
                /*age of business in months */
                    A.age_of_bus_in_mths,
                /*total expected monthly income amount */
                    A.tot_expct_mth_incm_amt,  
                /*total monthly income volatility amount*/
                    A.tot_mth_incm_vltl_amt,
                /*business annual turnover amount */
                    A.bus_annual_trnvr_amt,  
                    
                    
                /*****************************
                Strength of Credit Relationship
                *******************************/
                /*strength of relationship type*/
                    A.sor, 
                /*group strength of relationship type */
                    A.grp_sor, 
                /*Primary DDA Type */
                    A.prmry_dda_typ, 
                /*time with bank count */
                    A.tm_with_bnk_cnt,
                /*age of client relationship */
                    A.clnt_reltn_age_in_mths, 
                /*age of oldest credit relationship in months*/
                    A.age_oldest_cr_relt,
                /*time since last non-primary relationship */
                    A.tm_since_lst_non_prmry_reltn, 
                /*time/months since last strong strength of relationship*/
                    A.tm_mths_since_lst_strong_sor, 
                /*months since last address change*/
                    A.mth_since_lst_addr_chg, 
                /*customer relationship managment risk ranking*/
                    A.crm_rsk_rank,  
                    
                /**********************
                    Client Risk Index
                **********************/
                /*customer risk index display code*/
                    A.cri_disp_cd,
                /*client risk index scorecard identifier */
                    A.cri_scorcd_id,
                /*CRI raw score exlcude card data */
                    A.cri_raw_score_excl, 
                /*CRI raw score include card data*/
                    A.cri_raw_score_incl,
                /*client risk index general */
                    A.cri_gnrl,
                /*last month customer risk index */
                    A.lst_mth_cri, 
                /*noncheque rcl worst external behavioural score risk index */
                    A.non_chq_rcl_wrst_ext_bhvr_scor, 
                    
                /********************
                    Exposures
                ********************/
                /*client months 1-3 end of month minimum balance*/
                    A.m1_3_eom_min_bal, 
                /*client potential revolving credit exposure*/
                    A.potnl_rc_expsr,  
                /*client revolving credit exposure ratio mth1-12*/
                    A.clnt_revl_cr_expsr_rat_m1_12,  
                /*client unsecured exposure ratio mth1-12*/
                    A.clnt_unscty_expsr_rat_m1_12,  
                /*client actual exposure ignoring security*/
                    A.actl_expsr_ig_sec,  
                /*client actual exposure including security*/
                    A.actl_expsr_in_sec, 
                /*client actual exposure including mortgages*/
                    A.actl_expsr_in_mtge, 
                /*total unsecured lending amount*/
                    A.tot_unscty_lend_amt, 
                /*total secured lending amount */
                    A.tot_secure_lend_amt,
                /*total approved security value */
                    A.tot_aprov_scty_val,
                /*total exposure unsecured amount */
                    A.tot_expsr_unscty_amt, 
                /*total exposure unsecured factored */
                    A.tot_expsr_unscty_fct,
                /*unsecured lending to total exposure percent */
                    A.unscty_lend_to_tot_expsr_pct,
                /* secured lending to total security percent */
                    A.secure_lend_to_tot_scty_pct,
                /* total rbc unsecured exposure */
                    A.tot_rbc_unscty_expsr_amt,  
                    
                /**********************
                    Utilization
                **********************/
                /*open to buy amount*/
                    A.open_to_buy_amt, 
                    
                /************************
                   Delinquency history
                ************************/
                /* total delinquency count*/
                    A.tot_dlq_cnt,
                /* total write off amount */
                    A.tot_wrtof_amt, 
                /* client total arrears amount*/
                    A.tot_amt_arrears,
                /* client worst current arrears */
                    A.worst_curr_arrears,
                /* client months 1-12 total write off amount */
                    A.m1_12_tot_wo_amt, 
                /* client total write off less recovery value */
                    A.tot_wo_les_rec_val,
                /* months since last write-off */
                    A.mth_cnt_since_lst_writeoff,  	
                /* cu number times in collection*/
                    A.cu_no_times_in_clct, 
                /* cu months since last collection entry*/
                    A.cu_mth_since_lst_clct_entry, 
                /* cu months since last broken promise*/
                    A.cu_mth_since_lst_brkn_prms,  
                /* cu total number broken promises*/
                    A.cu_tot_no_brkn_prms,
                /* cu total months since last review */
                    A.cu_tot_mth_since_lst_rvw, 
                    
                /********************
                  Revenue Amount
                *********************/
                /* total fee income amount*/
                    A.tot_fee_incm_amt, 
                /* total interest income amount */
                    A.tot_int_incm_amt, 
                    
                /* Cost Amount*/
                    A.tot_costs_amt, 
                    
                    
                /*	-- Capability to Payback*/
                /*-- client months 1-12 best saving position*/
                    A.clnt_m1_12_bst_sav_posn,  
                /*-- client change balance to average deposit ratio*/
                    A.clnt_chg_bal_to_avg_dep_ratio, 
                /*-- client previous change balance to average deposit ratio */
                    A.clnt_prev_chg_bal_avg_dep_rat, 
                /* -- client yearly change balance to average deposit ratio */
                    A.clnt_yrly_chg_bal_avg_dep_rat,
                /* --client ratio business over consumer deposit turnover */
                    A.clnt_ratio_bus_over_cnsmr_dep, 
                /* --lending percent-total expected mthly income to total net revolving u*/
                    A.exp_mth_incm_to_rvl_unsec_lend, 
                /*-- last collections status code*/
                    A.lst_clctn_sts_cd, 
                /*-- last pre-collections triage status code */
                    A.lst_pre_clctn_triage_sts_cd,
                /*--number of months of probe history count */
                    A.no_of_mth_of_probe_hist_cnt,  
                    
                     
                /*	-- Personal Credit Information*/
                /*-- pc age oldest credit relationship*/
                    A.pc_age_oldest_cr_reltn, 
                /*-- pc last months cri value*/
                    A.pc_lst_mth_cri_val, 
                /* -- pc average months customer generated deposit month 1-12*/
                    A.pc_avg_mth_cg_dep_mth_1_12,
                /*-- pc expected value deposit month 1-6 joint*/
                    A.pc_expct_val_dep_mth_1_6_jnt,
                /* -- pc actual exposure including mortgage amount*/
                    A.pc_actl_expsr_incl_mortg_amt, 
                /*-- pc ratio business consumer deposit value*/
                    A.pc_rato_bus_cnsmr_dep_val, 
                /*--pc ttl rbc unsecured exposure amount */
                    A.pc_ttl_rbc_unscty_expsr_amt, 
                /*-- pc fl1 cri generated node identifier*/
                    A.pc_fl1_cri_gnrtd_node_id, 
                /*-- pc cri value*/
                    A.pc_cri_val,  
                /*--pc fl1 assign gbix node identifier*/
                    A.pc_fl1_asgn_gbix_node_id,
                /*-- pc gbix flag */
                    A.pc_gbix_flg, 
                /*-- pc cri display code*/
                    A.pc_cri_displ_cd, 
                /*-- pc cri crm code*/
                    A.pc_cri_crm_cd,  
                /*-- pc strength of relationship code*/
                    A.pc_sor_cd,  
                    
                /*	-- BusinessCredit Information*/
                /*-- bc age oldest cred relationship*/
                    A.bc_age_oldest_cr_reltn, 
                /*--bc age of business */
                    A.bc_age_of_bus, 
                /*-- bc last months cri value*/
                    A.bc_lst_mth_cri_val,
                /*-- bc average months customer generated deposit month 1-12 */
                    A.bc_avg_mth_cg_dep_mth_1_12,
                /*--bc expected value deposit month 1-6 joint */
                    A.bc_expct_val_dep_mth_1_6_joint,
                /*--bc actual exposure including mortgage amount */
                    A.bc_actl_expsr_incl_mortg_amt,
                /*-- bc ratio business consumer deposit value */
                    A.bc_rato_bus_cnsmr_dep_val,
                /* -- bc ttl rbc unsecured exposure amount */
                    A.bc_ttl_rbc_unscty_expsr_amt,
                /*-- bc fl1 cri generated node identifier */
                    A.bc_fl1_cri_gen_node_id,
                /* -- bc cri value */
                    A.bc_cri_val, 
                /*-- bc fl1 assign gbix node identifier*/
                    A.bc_fl1_asgn_gbix_node_id,
                /*  -- bc gbix flag */
                    A.bc_gbix_flg,
                /* -- bc cri display code*/
                    A.bc_cri_displ_cd, 
                /* --bc cri crm code*/
                    A.bc_cri_crm_cd, 
                /* --bc strength of relationship code*/
                    A.bc_sor_cd, 
                /* -- collections reason code*/
                    A.most_rcnt_clct_reason_code,
                /*  -- triage reason exit date*/
                    A.triage_reason_exit_date,
                /* -- triage reason exit code*/
                    A.triage_reason_exit_code,
                
                
                /*	---- The following are variables that were caputured in 2010 Business CRI model ------*/
                /*-- residential code*/
                    A.RESID_CD, 
                /*-- GEOGRAPHICAL UNIT */
                    A.GEO_UNT, 
                /*-- CLIENT ANY ACCOUNT CURRENTLY IN COLLECTIONS*/
                    A.ANY_ACCT_IN_CLCT, 
                /*-- CLIENT VULNERABILITY CODE */
                    A.CLNT_VULNRABLTY_CD,
                /*-- CLIENT LIFE TIME VALUE  */
                    A.CLNT_LIFE_TM_VAL, 
                
                
                /*	--- Credit Application Data*/
                /*-- CREDIT APPLICATION TOTAL DEBT SERVICE RATIO */
                    B.CA_TDS,  
                /*--CREDIT APPLICATION SERVICE TYPE CODE*/
                    B.CA_SRVC_TYP_CD,
                /*--CREDIT APPLICATION MOST RECENT PRODUCT TYPE  */
                    B.CA_MOST_RECNT_PROD_TYP ,
                /*-- CREDIT APPLICATION DELIVERY CHANNEL */
                    B.CA_DELVRY_CHNL,
                /*-- CREDIT APPLICATION MOST RECENT APPLICATION DATA AGE  */
                    B.CA_MST_RCNT_AP_AGE, 
                /*-- CREDIT APPLICATION HOUSING PAYMENT  */
                    B.CA_HOUSING_PYMNTS, 
                /* --CREDIT APPLICATION INCOME AMOUNT */
                    B.CA_INCM_AMT,
                /*-- CREDIT APPLICATION OVERRIDE REASON CODE BY CREDIT APPROVAL OFFICER */
                    B.CA_OVRD_REAS_CD_BY_CAO,
                /*--CREDIT APPLICATION LENDER OVERRIDE REASON CODE  */
                    B.CA_LN_OVRD_REAS_CD, 
                
                /*	---Credit Bureau Data*/
                /*--NUMBER BK/NATL TRADES W/BAL 75% HIGH CREDIT */
                    B.CB_BNK_NTL_TRD_75PCT_HICR_CNT, 
                /* --CREDIT BUREAU TOTAL LOAN COMMITMENT */
                    B.CB_TOT_LN_COMT, 
                /* -- CREDIT BUREAU MOST RECENT CREDIT BUREAU SCORE*/
                    B.MST_RECNT_CB_SCORE, 
                /*--HIGHEST CREDIT LINE UTILIZATION BANKCARD TRADELINE*/
                    B.CB_BNKCRD_TRADELINE_HI_CR_AMT,
                /*--MAXIMUM CREDIT LIMIT UTILIZATION ON EXTERNAL/BANKCARD/RETAIL CARD  */
                    B.CB_EXTRNL_BNKCRD_RTL_MAX_CRAMT, 
                /* --TOTAL ESTIMATED UNSECURED EXTERNAL BALANCES */
                    B.CB_TOT_EST_UNSECR_EXTRN_BAL,
                /*--CREDIT BUREAU WORST RATING EVER  */
                    B.CB_WRST_EVER_CR_BUR_RTG,
                /*-- CREDIT BUREAU TOTAL ALL REVOLVING LIMITS  */
                    B.CB_TOT_REVLV_LMTS,
                /*--CREDIT BUREAU OLDEST TRADE AGE  */
                    B.CB_OLDEST_TRD_AGE,
                /* --CREDIT BUREAU AVERAGE LAST 3 MONTHS BANK REVOLVING OUTSTANDING BALANCE */
                    B.CB_BNK_LST_3M_OS_B,
                /* -- CREDIT BUREAU EXCEPTION FLAG TYPE  */
                    B.CB_EXCPT_FLG, 
                /* -- CREDIT BUREAU LAST 12 MONTHS XRTL 30DAYS DELINQUENT COUNT */
                    B.CB_LST12MTH_XRTL_30DY_DLQ_CNT,
                /* -- NUMBER OF TRADES 30 DPD IN LAST 12 MONTHS  */
                    B.CB_LST_12MTH_TRADES_30_DLQ_CNT,
                /* -- NUMBER OF TRADES 60 DPD IN LAST 12 MONTHS */
                    B.CB_LST_12MTH_TRADES_60_DLQ_CNT,
                /*-- NUMBER OF TIMES 30 DPD IN LAST 12 MONTHS*/
                    B.CB_LST_12_MTH_30_DAY_DLQ_CNT,  
                /*  -- NUMBER OF TRADES 90+ DPD IN LAST 12 MONTHS */
                    B.CB_LST_12_MTH_90_DYS_DLQ_CNT,
                /*-- CREDIT BUREAU LAST 6 MONTHS WORST DELINQUENCY RATING ALL TRADES*/
                    B.CB_LST_6M_DELQ_RTG,
                /*-- CREDIT BUREAU WORST CURRENT DELINQUENCY RATING  */
                    B.CB_WORST_DELQ_RTG, 
                /* -- HIGHEST AMOUNT OF BANKCARD TRADE HC/CL ACTIVE*/
                    B.CB_HI_BNKCRD_HC_CL_ACTVTRD_AMT,
                /*  --AGE IN MONTHS OF OLDEST OPEN REVOLVING TRADE */
                    B.CB_OLDST_REVL_OPEN_TRD_AGE_MTH, 
                /*-- MONTHS SINCE MOST RECENT DELINQUENCY */
                    B.CB_MOST_RECNT_DLQ_IN_MTHS,
                /* -- CREDIT BUREAU AVG LAST 3 MONTHS REVOLVING OUTSTANDING BALANCE */
                    B.CB_LST_3M_OS_BAL,
                /* -- AVERAGE NUMBER OF MONTHS BANKCARDS ON FILE  */
                    B.CB_BNKCRD_AVG_MTH_CNT,
                /*   -- MONTHS SINCE OLDEST BANKCARD TRADE OPENED */
                    B.CB_OLDST_BNKCRD_OPEN_TRD_MTHS, 
                /* -- MONTHS SINCE MOST RECENT INQUIRY */
                    B.CB_MOST_RECNT_INQ_IN_MTHS,
                /*-- CREDIT BUREAU OPEN TO BUY */
                    B.CB_OPEN_TO_BUY, 
                /* -- NUMBER MONTHS SINCE LAST REPORTED 60 DPD */
                    B.CB_LST_60_DAY_DLQ_MTH_CNT,
                
                /*	-- More features added from CLNT_BHVR_CHARSTC*/
                /*-- Criteria Paper BRR*/
                    A.BRR, 
                /*-- Change in client risk index */
                    A.CHG_IN_CRI,
                /* -- Change in client group risk index */
                    A.CHG_IN_GRP_CRI_TYP1,
                /*-- Client profitability*/
                    A.CLNT_PROF, 
                /* -- CRI Customer relationship management code*/
                    A.CRI_CRM_CD,
                /*-- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 1*/
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT1,
                /*-- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 2 */
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT2, 
                /*-- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 3*/
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT3,
                /* -- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 4 */
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT4,
                /* -- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 5*/
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT5,
                /* -- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 6*/
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT6,
                /* -- CURRENT CREDIT TURNOVER TO UNSECURED LENDING PERCENT 7*/
                    A.CURR_CR_TRNVR_UNSCTY_LEND_PCT7,
                /*-- CUSTOMER GENERATED DEPOSITS M1-12*/
                    A.CUS_CGD_L12M, 
                /* -- CUSTOMER GENERATED DEPOSITS M1-3*/
                    A.CUS_CGD_L3M, 
                /*-- DISTRICT NUMBER*/
                    A.DIST_NO,
                /*-- Good/Bad/Indetermined Flag */
                    A.GBIX_FLG_TYP,
                /*-- GROUP CRI CUSTOMER RELATIONSHIP MANAGEMENT CODE */
                    A.GRP_CRI_CRM_CD, 
                /* -- GROUP CRI FLAG INDICATOR*/
                    A.GRP_CRI_FLG_IND, 
                /*-- GROUP CRI FLAG INDICATOR*/
                    A.GRP_CRI, 
                /*-- GROUP CUSTOMER RISK INDEX DISPLAY CODE*/
                    A.GRP_CRI_DISP_CD, 
                /*-- Group Key*/
                    A.GRP_KEY,  
                /*-- MONTHS SINCE LAST SIGNIFICANT INCOME COUNT*/
                    A.MTH_SINCE_LST_SIGT_INCM_CNT, 
                /* -- MONTHS WITH SIGNIFICANT INCOME L6M COUNT */
                    A.MTH_WITH_SIGT_INCM_L6M_CNT, 
                /*  -- POST RATING STRENGTH OF BUSINESS RELATIONSHIP*/
                    A.POST_RTG_STR_OF_BUS_RELTN,
                /*-- POST RATING STRENGTH OF CONSUMER RELATIONSHIP*/
                    A.POST_RTG_STR_OF_CNSMR_RELTN, 
                /* -- RESPONSIBILITY TRANSIT NUMBER */
                    A.RESP_TR_NO, 
                /*-- TIME/MONTHS SINCE LAST CONSOLIDATION */
                    A.TM_SINCE_LST_CNSLN,
                /*-- TOTAL ANNUAL INCOME 12 MONTHS AGO AMOUNT */
                    A.TOT_ANINCM_TO_ANN_INCM_12MAGO, 
                /* --TOTAL ANNUAL INCOME AMOUNT*/
                    A.TOT_ANN_INCM_AMT, 
                /* -- TOTAL ANNUAL INCOME CONSUMER TO BUSINESS PERCENT */
                    A.TOT_ANN_INCM_CNSMR_TO_BUS_PCT,
                /*-- TOTAL ANNUAL INCOME TO TOTAL ANNUAL INCOME 12M AGO */
                    A.TOT_ANN_INCM_12_MTH_AGO, 
                /* -- TOTAL EXPECT MTHLY CREDIT COMITMENT TO TOTAL EXPECT MNTHLY INCOME */
                    A.TOT_EXPCT_CRCOMT_MTH_INCM_PCT, 
                /*-- TOTAL EXPECTED MONTHLY CREDIT COMMITMENT AMOUNT*/
                    A.TOT_EXPCT_MTH_CR_COMT_AMT, 
                /* -- TOTAL EXPECTED MONTHLY INCOME TO TNUL */
                    A.TOT_EXPCT_MTH_INCM_TO_TNUL_PCT, 
                /*  -- TOTAL INCOME CM TO TOTAL EXPECTED MONTHLY INCOME PERCENT*/
                    A.TOT_INCM_CM_TO_EXPCT_INCM_PCT,
                /* -- TOTAL INCOME L3M TO L13-15M PERCENT*/
                    A.TOT_INCM_L3M_TO_L13_15M_PCT, 
                /* -- TOTAL MONTHLY INCOME 1 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_1_MTH_AGO_AMT, 
                /*-- TOTAL MONTHLY INCOME 10 MONTHS AGO AMOUNT*/
                    A.TOT_MTH_INCM_10_MTH_AGO_AMT, 
                /* -- TOTAL MONTHLY INCOME 11 MONTHS AGO AMOUNT*/
                    A.TOT_MTH_INCM_11_MTH_AGO_AMT, 
                /* -- TOTAL MONTHLY INCOME 11 MONTHS AGO AMOUNT */
                    A.TOT_MTH_INCM_12_MTH_AGO_AMT, 
                /* -- TOTAL MONTHLY INCOME 13 MONTHS AGO AMOUNT*/
                    A.TOT_MTH_INCM_13_MTH_AGO_AMT,
                /* -- TOTAL MONTHLY INCOME 14 MONTHS AGO AMOUNT */
                    A.TOT_MTH_INCM_14_MTH_AGO_AMT,
                /* -- TOTAL MONTHLY INCOME 15 MONTHS AGO AMOUNT */
                    A.TOT_MTH_INCM_15_MTH_AGO_AMT, 
                /* -- TOTAL MONTHLY INCOME 2 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_2_MTH_AGO_AMT, 
                /*-- TOTAL MONTHLY INCOME 3 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_3_MTH_AGO_AMT,  
                /* -- TOTAL MONTHLY INCOME 4 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_4_MTH_AGO_AMT, 
                /*-- TOTAL MONTHLY INCOME 5 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_5_MTH_AGO_AMT,  
                /*  -- TOTAL MONTHLY INCOME 6 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_6_MTH_AGO_AMT,
                /*  -- TOTAL MONTHLY INCOME 7 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_7_MTH_AGO_AMT,
                /*  -- TOTAL MONTHLY INCOME 8 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_8_MTH_AGO_AMT,
                /*  -- TOTAL MONTHLY INCOME 9 MONTH AGO AMOUNT*/
                    A.TOT_MTH_INCM_9_MTH_AGO_AMT,
                /* -- TOTAL NET REVOLVING UNSECURED LENDING AMOUNT*/
                    A.TOT_NET_REVL_UNSCTY_LEND_AMT, 
                /*-- TOTAL NET UNSECURED LENDING AMOUNT*/
                    A.TOT_NET_UNSCTY_LEND_AMT,  
                /*-- TOTAL RISK WEIGHTED APPROVE SECURITY TO TOTAL APPR SECURITY PERCENT*/
                    A.TOT_RSK_WGT_SCTY_TOT_APRV_SEC,
                /*-- TOTAL RISK WEIGHTED APPROVE SECURITY TO TOTAL SECURED LENDING PERCENT */
                    A.TOT_RSK_WGT_SCTY_TOTSECUR_LEND,  
                /* -- TOTAL UNAUTHORIZED FEE INCOME AMOUNT*/
                    A.TOT_UNAUTH_FEE_INCM_AMT,
                /* -- TOTAL UNAUTHORIZED INTEREST INCOME AMOUNT*/
                    A.TOT_UNAUTH_INT_INCM_AMT, 
                /*-- UNDELIVERABLE ADDRESS INDICATOR*/
                    A.UNDELIV_ADDR_IND, 
                /*-- Criteria Paper BRR */
                    A.CMRCL_CP_BRR,  
                /*-- non-sufficient fund indicator*/
                    A.NON_SUFF_FND_TYP_CD, 
                
                /*-- More features added from CLNT_CR_CHARSTC_VRB*/
                /*-- Additional bureau score for last month 1*/
                   B.ADNL_BUR_SCOR_LST_MTH_1_VAL, 
                /*-- Additional bureau score for last month 2*/
                   B.ADNL_BUR_SCOR_LST_MTH_2_VAL, 
                /*-- Additional bureau score for last month 3*/
                   B.ADNL_BUR_SCOR_LST_MTH_3_VAL, 
                /*-- Additional bureau score for last month 4*/
                   B.ADNL_BUR_SCOR_LST_MTH_4_VAL, 
                /*-- Additional bureau score for last month 5*/
                   B.ADNL_BUR_SCOR_LST_MTH_5_VAL, 
                /*-- application status code*/
                   B.CA_APP_STS_CD,  
                /*-- application product type in last 24 months*/
                   B.CA_APP_PROD_TYP_LST_24_MTH,  
                /* -- Credit application exception or caution code*/
                   B.CA_EXCPT_CAUT_CD,
                /* -- Credit application exception resason code*/
                   B.CA_EXCPT_REAS_CD,
                /*-- Credit application scorecard id*/
                   B.CA_PROD_SCRECRD_ID 
                
                
                        
                FROM	DDWV11.clnt_bhvr_charstc as A
                
                INNER JOIN (Select clnt_no, 
                        MTH_END_DT, 
                        TOT_SN_EXP 
                        FROM DG6V19.BUS_PRTFOL_CLNT WHERE TOT_SN_EXP >1000000) C
                ON A.clnt_no = C.clnt_no and A.capture_dt = C.MTH_END_DT
            
            
                LEFT JOIN  DDWV11.CLNT_CR_CHARSTC_VRB as B
                ON A.clnt_no = B.clnt_no and A.capture_dt = B.capture_dt
                   
                WHERE A.pers_bus_flg_typ = 'B' and A.capture_dt > '{start_snap_dt}' AND A.capture_dt < '{end_snap_dt}'
    """
    return pd.read_sql(qry, connection)

In [83]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_bhvr_char(start_snap_dt, end_snap_dt)
df1.shape

(460593, 216)

In [84]:
df1.to_csv(raw_data_dir+'bhvr_char_y1.csv', index=False)

In [85]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_bhvr_char(start_snap_dt, end_snap_dt)
df2.shape

(498065, 216)

In [86]:
df2.to_csv(raw_data_dir+'bhvr_char_y2.csv', index=False)

In [87]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_bhvr_char(start_snap_dt, end_snap_dt)
df3.shape

(530605, 216)

In [88]:
df3.to_csv(raw_data_dir+'bhvr_char_y3.csv', index=False)

In [89]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_bhvr_char(start_snap_dt, end_snap_dt)
df4.shape

(863901, 216)

In [90]:
df4.to_csv(raw_data_dir+'bhvr_char_y4.csv', index=False)

# BRR Extract

In [91]:
def pull_brr_extract(start_snap_dt, end_snap_dt):    
    qry = f"""\
    SELECT 
            /* This datapull uses RB_CLNT as the base population for pulling data */
            /* month end date */
                   C.SNAP_DT as mth_end_dt, 
            /* client number */
                   C.clnt_no , 
            /* client group, used to define commercial and small business clients */
            case
                        when (c.BSC in (190,192,200,202,250,490,590)
                                or (c.BSC in (310,500,502,504,505,510) and 
                                    (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL))
                             ) then 'COMM'

                        when (c.BSC in (550,554,555)
                                and (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL)
                             ) then 'AG'
                        when c.BSC in (330) then 'PUB'
                        when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                and c.CURR_CMI_NO in (100,101,102,103,
                                                        200,201,202,203, 
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503)
                             )then 'SB'
                        else 'OOO'
                    end as Clnt_Grp,
            /* response transit center*/
                   C.resp_unt AS resp_transit,
            /*BSC code */
                   C.BSC, 
            /*Current client management index*/
                   C.CURR_CMI_NO AS cmi,
            /*country*/
                   CASE
                     WHEN c.snap_dt LE '2007-09-30' THEN ''
                     WHEN AA.prov_cd IN ('AB','BC','MB','NB','NL','NS','NT','NU','ON','PE','QC','SK','YT') THEN 'Canada'
                     ELSE 'Others'
                   END AS country, 
            /*STATE/COUNTRY NAME*/
                   AA.st_cntry_nm, 
            /*SUB COUNTRY CODE OF RESIDENCE (SUB_CNTRY_CD)*/
                   AA.prov_cd, 
            /*Critieria Paper BORROWER RISK RATING (BRR) ( RSK_RTG )*/
                   CH.RSK_RTG as BRR, 
            /* Numerical version of BRR */
                CASE
                            WHEN CH.RSK_RTG = '1+' THEN 1
                            WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD = 'H' THEN 2
                            WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD = 'M'  THEN 3
                            WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD IS NULL THEN 3
                            WHEN CH.RSK_RTG = '1' AND CH.RSK_RTG_MOD = 'L' THEN 4
                            WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD = 'H' THEN 5
                            WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD = 'M' THEN 6
                            WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD IS NULL THEN 6
                            WHEN CH.RSK_RTG = '2+' AND CH.RSK_RTG_MOD = 'L' THEN 7
                            WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD = 'H' THEN 8
                            WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD = 'M' THEN 9
                            WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD IS NULL THEN 9
                            WHEN CH.RSK_RTG = '2' AND CH.RSK_RTG_MOD = 'L' THEN 10
                            WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD = 'H' THEN 11
                            WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD = 'M' THEN 12
                            WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD IS NULL THEN 12
                            WHEN CH.RSK_RTG = '2-' AND CH.RSK_RTG_MOD = 'L' THEN 13
                            WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD = 'H' THEN 14
                            WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD = 'M' THEN 15
                            WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD IS NULL THEN 15
                            WHEN CH.RSK_RTG = '3+' AND CH.RSK_RTG_MOD = 'L' THEN 16
                            WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD = 'H' THEN 17
                            WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD = 'M' THEN 18
                            WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD IS NULL THEN 18
                            WHEN CH.RSK_RTG = '3' AND CH.RSK_RTG_MOD = 'L' THEN 19
                            WHEN CH.RSK_RTG = '4' THEN 20
                            WHEN CH.RSK_RTG = '5' THEN 21
                            WHEN CH.RSK_RTG = '6' THEN 22
                          END as num_brr,
            /*Special Loan Indicator*/
                   case
                     when BB.clnt_no is not null then 'Y' /* if client exists in the BB table, then it is SLG client */
                     else 'N'
                     end as slg_ind,
            /* single name ID */
                     CASE
                           WHEN SN.SN_ID IS NULL THEN C.CLNT_NO
                           ELSE SN.SN_ID
                         END AS SN_ID,
            /* sic number */
                   CH.sic_cd1 AS sic,
            /* sic description */
                   S.sic_desc_en,
            /*CUSTOMER RISK INDEX GENERAL (CRI_GNRL)*/
                   D.cri_gnrl,
            /*DATE BUSINESS ESTABLISHED*/
                   B.dt_bus_estb,
            /*VERTICAL INDUSTRY CLASSIFICATION CODE*/
                   V.vic,  
            /*VERTICAL INDUSTRY CODE IDENTIFIER*/
                   V.vic_id,
            /*VERTICAL INDUSTRY CLASSIFICATION CODE DESCRIPTION*/
                   V.vic_desc_en, 
            /*SUB-VERTICAL INDUSTRY CLASSIFICATION CODE*/
                   V.sub_vic, 
            /*SUB-VERTICAL INDUSTRY CODE IDENTIFIER*/
                   V.sub_vic_id,
            /*SUB-VERTICAL INDUSTRY CLASSIFICATION CODE DESCRIPT*/
                   V.sub_vic_desc_en, 
            /* Personal/business customer type flag from Probe */
                    D.pers_bus_flg_typ,
            /* Date client signed up with RBC*/
                    ch.dt_opened,
            /* Number of years being RBC client */
                    (ch.mth_end_dt - ch.DT_OPENED)/365  AS yrs_being_client,
            /*BRR REVIEW DATE ( BRR_RVW_DT )*/
                   C.brr_rvw_dt AS brr_review_dt, 
            /* Client status (Active/Inactive) with 1 month lag*/
                   C.clnt_sts as status,
            /*Client Total Authorization*/
                   CASE WHEN BPC.clnt_auth is not null then BPC.clnt_auth
                        ELSE 0
                    end as clnt_auth, 
            /*Client Total Outstanding*/
                   CASE WHEN BPC.clnt_os is not null then BPC.clnt_os
                        ELSE 0
                    end as clnt_os,
            /*Client total single name exposure*/
                    CASE WHEN BPC.tot_sn_exp is not null then BPC.tot_sn_exp
                        ELSE 0
                    end as tot_sn_exp,
            /*Maximum Historical Total Single Name Exposure*/
                   case when T.max_tsne is not null then t.max_tsne
                   else 0 
                   end as max_tsne,
            /*CBRR without the L/M/H*/ 
                    C.CMRCL_BRR as CBRR,
            /* CBRR L/M/H modifier */
                    C.CMRCL_BRR_MOD_CD as CBRR_MOD_CD,
            /*numerical CBRR from EDW Table*/
                    CASE
                            WHEN C.CMRCL_BRR = '1+' THEN 1
                            WHEN C.CMRCL_BRR = '1' AND C.CMRCL_BRR_MOD_CD = 'H' THEN 2
                            WHEN C.CMRCL_BRR = '1' AND C.CMRCL_BRR_MOD_CD = 'M'  THEN 3
                            WHEN C.CMRCL_BRR = '1' AND C.CMRCL_BRR_MOD_CD IS NULL THEN 3
                            WHEN C.CMRCL_BRR = '1' AND C.CMRCL_BRR_MOD_CD = 'L' THEN 4
                            WHEN C.CMRCL_BRR = '2+' AND C.CMRCL_BRR_MOD_CD = 'H' THEN 5
                            WHEN C.CMRCL_BRR = '2+' AND C.CMRCL_BRR_MOD_CD = 'M' THEN 6
                            WHEN C.CMRCL_BRR = '2+' AND C.CMRCL_BRR_MOD_CD IS NULL THEN 6
                            WHEN C.CMRCL_BRR = '2+' AND C.CMRCL_BRR_MOD_CD = 'L' THEN 7
                            WHEN C.CMRCL_BRR = '2' AND C.CMRCL_BRR_MOD_CD = 'H' THEN 8
                            WHEN C.CMRCL_BRR = '2' AND C.CMRCL_BRR_MOD_CD = 'M' THEN 9
                            WHEN C.CMRCL_BRR = '2' AND C.CMRCL_BRR_MOD_CD IS NULL THEN 9
                            WHEN C.CMRCL_BRR = '2' AND C.CMRCL_BRR_MOD_CD = 'L' THEN 10
                            WHEN C.CMRCL_BRR = '2-' AND C.CMRCL_BRR_MOD_CD = 'H' THEN 11
                            WHEN C.CMRCL_BRR = '2-' AND C.CMRCL_BRR_MOD_CD = 'M' THEN 12
                            WHEN C.CMRCL_BRR = '2-' AND C.CMRCL_BRR_MOD_CD IS NULL THEN 12
                            WHEN C.CMRCL_BRR = '2-' AND C.CMRCL_BRR_MOD_CD = 'L' THEN 13
                            WHEN C.CMRCL_BRR = '3+' AND C.CMRCL_BRR_MOD_CD = 'H' THEN 14
                            WHEN C.CMRCL_BRR = '3+' AND C.CMRCL_BRR_MOD_CD = 'M' THEN 15
                            WHEN C.CMRCL_BRR = '3+' AND C.CMRCL_BRR_MOD_CD IS NULL THEN 15
                            WHEN C.CMRCL_BRR = '3+' AND C.CMRCL_BRR_MOD_CD = 'L' THEN 16
                            WHEN C.CMRCL_BRR = '3' AND C.CMRCL_BRR_MOD_CD = 'H' THEN 17
                            WHEN C.CMRCL_BRR = '3' AND C.CMRCL_BRR_MOD_CD = 'M' THEN 18
                            WHEN C.CMRCL_BRR = '3' AND C.CMRCL_BRR_MOD_CD IS NULL THEN 18
                            WHEN C.CMRCL_BRR = '3' AND C.CMRCL_BRR_MOD_CD = 'L' THEN 19
                            WHEN C.CMRCL_BRR = '4' THEN 20
                            WHEN C.CMRCL_BRR = '5' THEN 21
                            WHEN C.CMRCL_BRR = '6' THEN 22
                          END as NUM_CBRR,
            /* Newton BRR without the L/M/H */ 
                    C.NEWTON_BRR as CP_BRR,
            /* Newton BRR L/M/H modifier */
                    C.NEWTON_BRR_MOD_CD as CP_BRR_MOD_CD,
            /* numerical Newton BRR from EDW table */
                            CASE
                            WHEN C.NEWTON_BRR = '1+' THEN 1
                            WHEN C.NEWTON_BRR = '1' AND C.NEWTON_BRR_MOD_CD = 'H' THEN 2
                            WHEN C.NEWTON_BRR = '1' AND C.NEWTON_BRR_MOD_CD = 'M'  THEN 3
                            WHEN C.NEWTON_BRR = '1' AND C.NEWTON_BRR_MOD_CD IS NULL THEN 3
                            WHEN C.NEWTON_BRR = '1' AND C.NEWTON_BRR_MOD_CD = 'L' THEN 4
                            WHEN C.NEWTON_BRR = '2+' AND C.NEWTON_BRR_MOD_CD = 'H' THEN 5
                            WHEN C.NEWTON_BRR = '2+' AND C.NEWTON_BRR_MOD_CD = 'M' THEN 6
                            WHEN C.NEWTON_BRR = '2+' AND C.NEWTON_BRR_MOD_CD IS NULL THEN 6
                            WHEN C.NEWTON_BRR = '2+' AND C.NEWTON_BRR_MOD_CD = 'L' THEN 7
                            WHEN C.NEWTON_BRR = '2' AND C.NEWTON_BRR_MOD_CD = 'H' THEN 8
                            WHEN C.NEWTON_BRR = '2' AND C.NEWTON_BRR_MOD_CD = 'M' THEN 9
                            WHEN C.NEWTON_BRR = '2' AND C.NEWTON_BRR_MOD_CD IS NULL THEN 9
                            WHEN C.NEWTON_BRR = '2' AND C.NEWTON_BRR_MOD_CD = 'L' THEN 10
                            WHEN C.NEWTON_BRR = '2-' AND C.NEWTON_BRR_MOD_CD = 'H' THEN 11
                            WHEN C.NEWTON_BRR = '2-' AND C.NEWTON_BRR_MOD_CD = 'M' THEN 12
                            WHEN C.NEWTON_BRR = '2-' AND C.NEWTON_BRR_MOD_CD IS NULL THEN 12
                            WHEN C.NEWTON_BRR = '2-' AND C.NEWTON_BRR_MOD_CD = 'L' THEN 13
                            WHEN C.NEWTON_BRR = '3+' AND C.NEWTON_BRR_MOD_CD = 'H' THEN 14
                            WHEN C.NEWTON_BRR = '3+' AND C.NEWTON_BRR_MOD_CD = 'M' THEN 15
                            WHEN C.NEWTON_BRR = '3+' AND C.NEWTON_BRR_MOD_CD IS NULL THEN 15
                            WHEN C.NEWTON_BRR = '3+' AND C.NEWTON_BRR_MOD_CD = 'L' THEN 16
                            WHEN C.NEWTON_BRR = '3' AND C.NEWTON_BRR_MOD_CD = 'H' THEN 17
                            WHEN C.NEWTON_BRR = '3' AND C.NEWTON_BRR_MOD_CD = 'M' THEN 18
                            WHEN C.NEWTON_BRR = '3' AND C.NEWTON_BRR_MOD_CD IS NULL THEN 18
                            WHEN C.NEWTON_BRR = '3' AND C.NEWTON_BRR_MOD_CD = 'L' THEN 19
                            WHEN C.NEWTON_BRR = '4' THEN 20
                            WHEN C.NEWTON_BRR = '5' THEN 21
                            WHEN C.NEWTON_BRR = '6' THEN 22
                          END as NUM_CP_BRR, 
            /* EDW effective BRR without the L/M/H */ 
                    C.BRR as EFF_BRR,
            /* EDW effective BRR L/M/H modifier */
                    C.BRR_MOD_CD as EFF_BRR_MOD_CD,
            /* numerical Newton BRR from EDW table */
                            CASE
                            WHEN C.BRR = '1+' THEN 1
                            WHEN C.BRR = '1' AND C.BRR_MOD_CD = 'H' THEN 2
                            WHEN C.BRR = '1' AND C.BRR_MOD_CD = 'M'  THEN 3
                            WHEN C.BRR = '1' AND C.BRR_MOD_CD IS NULL THEN 3
                            WHEN C.BRR = '1' AND C.BRR_MOD_CD = 'L' THEN 4
                            WHEN C.BRR = '2+' AND C.BRR_MOD_CD = 'H' THEN 5
                            WHEN C.BRR = '2+' AND C.BRR_MOD_CD = 'M' THEN 6
                            WHEN C.BRR = '2+' AND C.BRR_MOD_CD IS NULL THEN 6
                            WHEN C.BRR = '2+' AND C.BRR_MOD_CD = 'L' THEN 7
                            WHEN C.BRR = '2' AND C.BRR_MOD_CD = 'H' THEN 8
                            WHEN C.BRR = '2' AND C.BRR_MOD_CD = 'M' THEN 9
                            WHEN C.BRR = '2' AND C.BRR_MOD_CD IS NULL THEN 9
                            WHEN C.BRR = '2' AND C.BRR_MOD_CD = 'L' THEN 10
                            WHEN C.BRR = '2-' AND C.BRR_MOD_CD = 'H' THEN 11
                            WHEN C.BRR = '2-' AND C.BRR_MOD_CD = 'M' THEN 12
                            WHEN C.BRR = '2-' AND C.BRR_MOD_CD IS NULL THEN 12
                            WHEN C.BRR = '2-' AND C.BRR_MOD_CD = 'L' THEN 13
                            WHEN C.BRR = '3+' AND C.BRR_MOD_CD = 'H' THEN 14
                            WHEN C.BRR = '3+' AND C.BRR_MOD_CD = 'M' THEN 15
                            WHEN C.BRR = '3+' AND C.BRR_MOD_CD IS NULL THEN 15
                            WHEN C.BRR = '3+' AND C.BRR_MOD_CD = 'L' THEN 16
                            WHEN C.BRR = '3' AND C.BRR_MOD_CD = 'H' THEN 17
                            WHEN C.BRR = '3' AND C.BRR_MOD_CD = 'M' THEN 18
                            WHEN C.BRR = '3' AND C.BRR_MOD_CD IS NULL THEN 18
                            WHEN C.BRR = '3' AND C.BRR_MOD_CD = 'L' THEN 19
                            WHEN C.BRR = '4' THEN 20
                            WHEN C.BRR = '5' THEN 21
                            WHEN C.BRR = '6' THEN 22
                          END as NUM_EFF_BRR,

            /* BRR from Probe */
                    D.BRR as BRR_FROM_PROBE,
                    D.NUM_BRR as NUM_BRR_FROM_PROBE,
                    D.TTC_BRR_RAW_1M_LAG as TTC_CBRR_1M_LAG_FROM_PROBE,
                    D.TTC_BRR_NO_1M_LAG as TTC_CBRR_NO_1M_LAG_FROM_PROBE,
                    D.CMRCL_CP_BRR as CMRCL_CP_BRR_FROM_PROBE


            FROM DDWV01.rb_clnt AS C

                    /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                 INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                      ON  r.SNAP_DT  = c.snap_dt
                          AND r.ORG_UNT_ID = c.resp_unt
                          AND r.PARNT_HIER_TYP_ID = 80000 
                          AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)

                /* Identify clients with SLG manager, logic obtained from Business Portfolio Client table source code */
                 LEFT JOIN
                 (
                     select distinct h.clnt_no, h.assign_typ, h.acct_mgr_cd,h.mth_end_dt
                     from dg6v19.clnt_am_assignment_hist as H
                     inner join ddwv01.org_unt_to_slg_mgr_reltn as S
                     on H.acct_mgr_cd = S.slg_tr_no
                     and H.mth_end_dt > '{start_snap_dt}' AND H.mth_end_dt < '{end_snap_dt}'
                     and H.assign_typ = 'S'
                      ) as BB
                        ON C.clnt_no = BB.clnt_no
                        and C.snap_dt = BB.mth_end_dt

                /* Used to get client SN ID, logic obtained from Business Portfolio Client table source code  */
                LEFT JOIN DG6V19.CLNT_SN_HIST as SN
                             ON C.CLNT_NO = SN.CLNT_NO
                            AND C.snap_DT = SN.MTH_END_DT

                /* use this table to get the link to address, VIC tables, SIC tables */
                 LEFT JOIN DG6V19.clnt_hist AS CH
                   ON CH.clnt_no = C.clnt_no
                      AND CH.mth_end_dt = c.snap_dt

                /* SIC info */
                 LEFT JOIN DDWV01.sic AS S
                   ON S.sic_cd = CH.sic_cd1
                      AND S.snap_dt = CH.mth_end_dt

                /* Business established date */
                 LEFT JOIN DG6V19.non_pers_clnt_hist B
                   ON B.clnt_no = C.clnt_no
                      AND B.mth_end_dt = c.snap_dt

                /* Address */
                 LEFT JOIN DG6V19.addr_hist AS AA
                   ON AA.addr_id = CH.addr_id
                      AND AA.mth_end_dt = CH.mth_end_dt
                      AND AA.addr_src = 'C'

                /* VIC */	
                 LEFT JOIN DG6V19.sub_vic_hist AS V
                   ON V.sub_vic_id = CH.sub_vic_id
                      AND V.mth_end_dt = CH.mth_end_dt  

                /* Probe table */
                 LEFT JOIN DDWV11.clnt_bhvr_charstc AS D
                   ON D.clnt_no = C.clnt_no
                      AND D.capture_dt = c.snap_dt 
                      AND D.pers_bus_flg_typ = 'B'

                /* table to identify if client has overdraft facility */
                left join ( select clnt_no, mth_end_dt, max(od_ind) as od_ind /* aggregate to client level */
                            from 
                            (select lend.clnt_no, /* arrangement level table */
                                  lend.mth_end_dt,
                                  lend.ar_id,
                                  CASE WHEN FAC_TABLE.FAC_TYPE1 in ('OD', 'OV', 'OM') THEN 1
                                    ELSE 0
                                  END AS od_ind
                                /* Arrangement ownership history table to link accounts to clients*/
                                  from DG6V19.ARNGMNT_OWN_HIST as lend 
                                  /* Facility table to identify overdraft facilities */
                                  inner join DG6V19.BUS_COMM_RPT_FAC as FAC_TABLE
                                  on FAC_TABLE.prod_no = lend.ar_id
                                  and FAC_TABLE.SNAP_DT = lend.MTH_END_DT
                                  where FAC_TABLE.SNAP_DT > '{start_snap_dt}' AND FAC_TABLE.SNAP_DT < '{end_snap_dt}'                                 
                                  ) as OD_fac 
                                group by 1, 2) as OD
                      ON C.CLNT_NO = OD.CLNT_NO 
                      AND c.snap_dt = OD.MTH_END_DT

                 /* Calculate max TSNE */
                 LEFT JOIN (SELECT t1.MTH_END_DT,  
                                    t1.CLNT_NO, 
                                    Max(t2.TOT_SN_EXP) AS Max_TSNE 
                             FROM 
                                    DG6V19.BUS_PRTFOL_CLNT AS t1,
                                    DG6V19.BUS_PRTFOL_CLNT AS t2
                             WHERE
                                    t1.CLNT_NO = t2.CLNT_NO AND 
                                    t1.MTH_END_DT >= t2.MTH_END_DT 
                             GROUP BY 1,2 )AS T

                   ON C.CLNT_NO = T.CLNT_NO 
                      AND c.snap_dt = T.MTH_END_DT

                /* business portfolio client table with client authorization, outstanding, max TSNE info */
                 LEFT JOIN DG6V19.BUS_PRTFOL_CLNT AS BPC
                   ON BPC.clnt_no = C.clnt_no
                      AND BPC.mth_end_dt = c.snap_dt

            WHERE CLNT_GRP is in ('COMM', 'AG', 'PUB') /* keep only commercial clients*/
            and (bpc.tot_sn_exp > 1000000) /* TSNE >1000000 */
            and (c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}')
            and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
            and c.clnt_typ=2 /* keep only non-personal clients */
            """
    return pd.read_sql(qry, connection)

In [92]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_brr_extract(start_snap_dt, end_snap_dt)
df1.shape

(447726, 46)

In [93]:
df1.to_csv(raw_data_dir+'brr_extract_y1.csv', index=False)

In [94]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_brr_extract(start_snap_dt, end_snap_dt)
df2.shape

(470041, 46)

In [95]:
df2.to_csv(raw_data_dir+'brr_extract_y2.csv', index=False)

In [96]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_brr_extract(start_snap_dt, end_snap_dt)
df3.shape

(497862, 46)

In [97]:
df3.to_csv(raw_data_dir+'brr_extract_y3.csv', index=False)

In [98]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_brr_extract(start_snap_dt, end_snap_dt)
df4.shape

(807177, 46)

In [99]:
df4.to_csv(raw_data_dir+'brr_extract_y4.csv', index=False)

# BDA TXN NO COBC

In [100]:
def pull_bda_txn_no_cobc(start_snap_dt, end_snap_dt):    
    bda_txn_no_cobc_qry = f"""\
    SELECT      
         A.mth_end_dt,

         A.ar_id,
         max(A.clnt_no),
         SUM(X.dr) AS dr_cnt,
         SUM( T.amt * X.dr) AS dr_amt
         ,
         MAX (
             CASE WHEN X.dr = 1 THEN T.amt  
             ELSE 0 END
          ) AS dr_max_amt,

         SUM(X.cr) AS cr_cnt,
         SUM( T.amt * X.cr) AS cr_amt,
         MAX (
            CASE WHEN X.cr = 1 THEN T.amt  
            ELSE 0 END
          ) AS cr_max_amt,

         SUM(X.nsf_cr) AS nsf_cr_cnt,
         SUM( T.amt * X.nsf_cr) AS nsf_cr_amt,
         MAX (
            CASE WHEN X.nsf_cr = 1 THEN T.amt  
            ELSE 0 END
            ) AS nsf_cr_max_amt,

          SUM(X.nsf_dr) AS nsf_dr_cnt,
          SUM( T.amt * X.nsf_dr) AS nsf_dr_amt,
          MAX (
             CASE WHEN X.nsf_dr = 1 THEN T.amt  
             ELSE 0 END
             ) AS nsf_dr_max_amt,

          SUM(X.nsf_fees) AS nsf_fees_cnt,
          SUM( T.amt * X.nsf_fees) AS nsf_fees_amt,
          MAX (
             CASE WHEN X.nsf_fees = 1 THEN T.amt  
             ELSE 0 END
            ) AS nsf_fees_max_amt,

          SUM(X.ret_itms) AS ret_itms_cnt,
          SUM( T.amt * X.ret_itms) AS ret_itms_amt,
          MAX (
             CASE WHEN X.ret_itms = 1 THEN T.amt  
             ELSE 0 END
           ) AS ret_itms_max_amt,

           COUNT(*) AS cnt 
    FROM 
      ddwv01.curr_acct_txn  AS T, 
     (
         select  txn_cd, 
                      dr_or_cr, 
                      CASE
                         WHEN dr_or_cr = 'dr' THEN 1
                         ELSE 0
                      END AS dr,

                      CASE
                          WHEN dr_or_cr = 'cr' THEN 1
                           ELSE 0
                       END AS cr,

                      CASE
                          WHEN txn_cd IN (128,135,140,202, 225, 503, 506, 507, 613,615) THEN 1
                          ELSE 0
                      END AS nsf_cr,

                      CASE
                          WHEN txn_cd IN (156,167) THEN 1
                          ELSE 0
                      END AS nsf_dr,

                      CASE
                          WHEN txn_cd IN (288,496,986) THEN 1
                          ELSE 0
                      END AS nsf_fees,

                      CASE
                          WHEN txn_cd IN (168,170,250, 261,299,423,589,593,594) THEN 1
                          ELSE 0
                      END AS ret_itms

                 FROM (SELECT txn_cd, dr_or_cr
                                FROM  DL_proews.vt_txn_cr_dr
                               ) AS A

       ) AS X,
       DL_proews.vt_client_core_no_cobc AS A

    /*    INNER JOIN (Select clnt_no, MTH_END_DT, TOT_SN_EXP 
        FROM DG6V19.BUS_PRTFOL_CLNT WHERE TOT_SN_EXP > 1000000) AS T1
        ON EXTRACT (YEAR FROM T1.MTH_END_DT) =  EXTRACT (YEAR FROM T.pst_dt) 
        AND EXTRACT (MONTH FROM T1.MTH_END_DT) =  EXTRACT (MONTH FROM T.pst_dt)
        AND A.clnt_no = T1.clnt_no */

    WHERE  T.txn_cd = X.txn_cd  
           AND   T.ar_id = A.ar_id
           AND EXTRACT (YEAR FROM A.mth_end_dt) =  EXTRACT (YEAR FROM T.pst_dt) 
           AND EXTRACT (MONTH FROM A.mth_end_dt) =  EXTRACT (MONTH FROM T.pst_dt)
           AND A.Clnt_Grp NOT IN ('OOO','SB')
    /*--  AND T.ar_id = '00000000009121003821'*/
      AND A.mth_end_dt > '{start_snap_dt}' AND A.mth_end_dt < '{end_snap_dt}'
      and T.pst_cd IN (1,2,3,4,901,902,903,904)
      GROUP BY 1,2;
    """
    return pd.read_sql(bda_txn_no_cobc_qry, connection)

In [101]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_bda_txn_no_cobc(start_snap_dt, end_snap_dt)
df1.shape

(2576061, 22)

In [102]:
df1.to_csv(raw_data_dir+'bda_txn_no_cobc_prod_lvl_y1.csv', index=False)

In [103]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_bda_txn_no_cobc(start_snap_dt, end_snap_dt)
df2.shape

(2593908, 22)

In [104]:
df2.to_csv(raw_data_dir+'bda_txn_no_cobc_prod_lvl_y2.csv', index=False)

In [105]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_bda_txn_no_cobc(start_snap_dt, end_snap_dt)
df3.shape

(2609344, 22)

In [106]:
df3.to_csv(raw_data_dir+'bda_txn_no_cobc_prod_lvl_y3.csv', index=False)

In [107]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_bda_txn_no_cobc(start_snap_dt, end_snap_dt)
df4.shape

(4016286, 22)

In [108]:
df4.to_csv(raw_data_dir+'bda_txn_no_cobc_prod_lvl_y4.csv', index=False)

# Loan General

In [109]:
def pull_loan_general(start_snap_dt, end_snap_dt):    
    loan_general_qry = f"""\
            select 
            /* month end date */
                   C.SNAP_DT as mth_end_dt, 
            /* client number */
                   C.clnt_no , 
            /* client group, used to define commercial and small business clients */
            case
                        when (c.BSC in (190,192,200,202,250,490,590)
                                or (c.BSC in (310,500,502,504,505,510) and 
                                    (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203,
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL))
                             ) then 'COMM'
                
                        when (c.BSC in (550,554,555)
                                and (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL)
                             ) then 'AG'
                        when c.BSC in (330) then 'PUB'
                        when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                and c.CURR_CMI_NO in (100,101,102,103,
                                                        200,201,202,203,
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503)
                             )then 'SB'
                        else 'OOO'
                    end as Clnt_Grp,
                  A.ar_id, /*-- arrangement id*/
                  'A' || trim(A.ar_id) as t_ar_id, /* Added by QH 2018-11-08*/
                  A.dw_srvc_id,  /*-- dw service id*/
                  A.dfr_pymt_ind,  /*-- deferred payment indicator*/
                  A.dfr_pymts_tot, /*-- defered payment total*/
                  A.delay_pymt_due_dt, /*-- delayed payments total*/
                  A.ov_lmt_alwnc_amt, /*-- over limit allowance amount*/
                  A.pymt_retry_cntr, /*-- payment retry counter*/
                  A.number_waived, /*-- total number of payment waived*/
                  L.mif_srvc_id/* -- MIF service id*/
                
            
            FROM DDWV01.rb_clnt AS C
            
                    INNER JOIN (Select clnt_no, MTH_END_DT, TOT_SN_EXP 
                    FROM DG6V19.BUS_PRTFOL_CLNT WHERE TOT_SN_EXP > 1000000) AS T
                    ON c.snap_dt = T.MTH_END_DT
                    AND C.clnt_no = T.clnt_no

                    
                    /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                 INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                      ON  r.SNAP_DT  = c.snap_dt
                          AND r.ORG_UNT_ID = c.resp_unt
                          AND r.PARNT_HIER_TYP_ID = 80000 
                          AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)
                               
                 INNER JOIN DG6V19.bus_prtfol_lend as L
                     on L.mth_end_dt = C.snap_dt
                        and L.clnt_no = C.clnt_no
                        /*--and L.mth_end_dt LE '2018-05-31'*/
                        and L.mif_srvc_id = 20
            
                 INNER JOIN DDWV01.ln_general as A
                     on A.snap_dt = L.mth_end_dt
                        and A.ar_id = L.ar_id 
                        and A.dw_srvc_id = L.mif_srvc_id 
                        and A.dw_srvc_id = 20
            /*			--and A.snap_dt LE '2018-05-31'*/
            
            where clnt_grp NOT IN ('OOO') and clnt_grp NOT IN ('SB') 
                  and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                  and c.clnt_typ=2 /* keep only non-personal clients */
                and c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'
            """
    return pd.read_sql(loan_general_qry, connection)

In [110]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_loan_general(start_snap_dt, end_snap_dt)
df1.shape

(1089142, 13)

In [111]:
df1.to_csv(raw_data_dir+'loan_general_y1.csv', index=False)

In [112]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_loan_general(start_snap_dt, end_snap_dt)
df2.shape

(1136590, 13)

In [113]:
df2.to_csv(raw_data_dir+'loan_general_y2.csv', index=False)

In [114]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_loan_general(start_snap_dt, end_snap_dt)
df3.shape

(1207142, 13)

In [115]:
df3.to_csv(raw_data_dir+'loan_general_y3.csv', index=False)

In [116]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_loan_general(start_snap_dt, end_snap_dt)
df4.shape

(1963653, 13)

In [117]:
df4.to_csv(raw_data_dir+'loan_general_y4.csv', index=False)

# Agg Loan TXN Acct Level

In [ ]:
def pull_agg_loan_txn(start_snap_dt, end_snap_dt):    
    aggr_loan_txn_qry = f"""\
        Select 
        a.mth_end_dt, 
        a.CLNT_NO, 
        a.clnt_grp,
        a.AR_ID, 
        a.t_ar_id,
        a.CRNCY_CD,

        count(a.TXN_CD) as num_txn,       /* number of transactions made in a month*/
                                         /* TO DO: most-frequent transaction code in a month*/
        sum(a.AMT) as tot_amt,
        max(a.AMT) as max_amt,
        min(a.AMT) as min_amt,
        avg(a.AMT) as avg_amt, 
        stddev_samp(a.AMT) as vol_amt, /* standard deviation, i.e., volatility amt in a month*/

        sum(a.PRNCPL_AMT) as tot_prncpl_amt,
        max(a.PRNCPL_AMT) as max_prncpl_amt, 
        min(a.PRNCPL_AMT) as min_prncpl_amt,
        avg(a.PRNCPL_AMT) as avg_prncpl_amt,
        stddev_samp(a.PRNCPL_AMT) as vol_prncpl_amt,

        sum(a.PREPAY_AMT) as tot_prepay_amt,
        max(a.PREPAY_AMT) as max_prepay_amt,
        min(a.PREPAY_AMT) as min_prepay_amt,
        avg(a.PREPAY_AMT) as avg_prepay_amt, 
        stddev_samp(a.PREPAY_AMT) as vol_prepay_amt,

        sum(a.INT_AMT) as tot_int_amt,
        max(a.INT_AMT) as max_int_amt,
        min(a.INT_AMT) as min_int_amt,
        avg(a.INT_AMT) as avg_int_amt, 
        stddev_samp(a.INT_AMT) vol_int_amt

        FROM (
        SELECT 
            /* month end date */
                   C.SNAP_DT as mth_end_dt, 
            /* client number */
                   C.clnt_no , 
            /* client group, used to define commercial and small business clients */
            case
                        when (c.BSC in (190,192,200,202,250,490,590)
                                or (c.BSC in (310,500,502,504,505,510) and 
                                    (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL))
                             ) then 'COMM'
                
                        when (c.BSC in (550,554,555)
                                and (c.CURR_CMI_NO not in (100,101,102,103,
                                                            200,201,202,203, 
                                                            300,301,302,303,
                                                            401,402,403,
                                                            501,502,503) or c.CURR_CMI_NO is NULL)
                             ) then 'AG'
                        when c.BSC in (330) then 'PUB'
                        when (c.BSC in (310,500,502,504,505,510, 550,554,555) 
                                and c.CURR_CMI_NO in (100,101,102,103,
                                                        200,201,202,203,
                                                        300,301,302,303,
                                                        401,402,403,
                                                        501,502,503)
                             )then 'SB'
                        else 'OOO'
                    end as Clnt_Grp,
            /*-- arrangement id*/
                   L.ar_id , 
             'A' || trim(L.ar_id) as t_ar_id, /* Added by QH 2018-11-08*/
            /*-- mif service id*/
                   L.mif_srvc_id, 
            /*-- loan discription*/
                   L.ln_desc , 
            /*-- snapshot date*/
                   X.snap_dt,
            /*-- effective date*/
                   X.eff_dt,
            /*-- currency code */
                   A.crncy_cd, 
            /*-- transaction status code*/
                   X.txn_sts_cd, 
            /* -- transaction code*/
                   X.txn_cd,  
            /* -- correction indicator*/
                   X.correction_ind,
            /*-- acount */
                   X.amt,  
            /*-- principal amount*/
                   X.prncpl_amt,  
            /*-- prepayment amount*/
                   X.prepay_amt, 
            /*-- interest amount*/
                   X.int_amt, 
            /*-- transaction fee amount*/
                   X.txn_fee_amt, 
            /*-- transaction insurance amount */
                   X.txn_ins_amt  
            
            
            FROM DDWV01.rb_clnt AS C	
            
                    /* Inner join with roll-up rank hierarchy table to get Canadian banking clients */
                 INNER JOIN DDWV01.ROLL_HIER_RELTN_RNK AS R
                      ON  r.SNAP_DT  = c.snap_dt
                          AND r.ORG_UNT_ID = c.resp_unt
                          AND r.PARNT_HIER_TYP_ID = 80000 
                          AND r.ROLL_ID_RANK_600 IN (80013,80029,80146,82314,83080,83081,83082,83091)
				
				INNER JOIN (Select clnt_no, 
                        MTH_END_DT as snap_dt, 
                        TOT_SN_EXP 
                        FROM DG6V19.BUS_PRTFOL_CLNT WHERE TOT_SN_EXP >1000000) B /*filter out single name exposure 1MM*/
                ON C.clnt_no = B.clnt_no and C.snap_dt = B.snap_dt
            
                  INNER JOIN DG6V19.bus_prtfol_lend AS L
                     ON L.clnt_no = C.clnt_no
                         AND L.mth_end_dt = C.snap_dt
            
            
                  INNER JOIN DG6V19.arngmnt_hist AS A
                     ON A.ar_id = L.ar_id
                        AND A.mth_end_dt = L.mth_end_dt
                        AND A.mif_srvc_id = L.mif_srvc_id
					
            
            
                  INNER JOIN DDWV01.ln_mrtg_prncpl_nt AS X /*mif_srvc_id 20 and 21 only */
                    ON X.ar_id = L.ar_id
                       AND X.srvc_cd = L.mif_srvc_id
                       AND Extract (YEAR From X.snap_dt) = Extract (YEAR From L.mth_end_dt)
                       AND Extract (MONTH From X.snap_dt) = Extract (MONTH From L.mth_end_dt)
            
            where clnt_grp NOT IN ('OOO', 'SB') 
                  and (c.clnt_sts='A' or c.clnt_sts is null) /* filter out inactive clients */
                  and c.clnt_typ=2 /* keep only non-personal clients */
            and c.snap_dt > '{start_snap_dt}' AND c.snap_dt < '{end_snap_dt}'
        ) a
        group by a.mth_end_dt, a.clnt_no, a.clnt_grp, a.AR_ID, a.t_ar_id, a.CRNCY_CD
        """
    return pd.read_sql(aggr_loan_txn_qry, connection)

In [ ]:
start_snap_dt='2015-01-01'
end_snap_dt='2016-01-01'
df1 = pull_agg_loan_txn(start_snap_dt, end_snap_dt)
df1.shape

In [111]:
df1.to_csv(raw_data_dir+'agg_loan_txn_acct_lvl_y1.csv', index=False)

In [ ]:
start_snap_dt='2016-01-01'
end_snap_dt='2017-01-01'
df2 = pull_agg_loan_txn(start_snap_dt, end_snap_dt)
df2.shape

In [ ]:
df2.to_csv(raw_data_dir+'agg_loan_txn_acct_lvl_y2.csv', index=False)

In [ ]:
start_snap_dt='2017-01-01'
end_snap_dt='2018-01-01'
df3 = pull_agg_loan_txn(start_snap_dt, end_snap_dt)
df3.shape

In [ ]:
df3.to_csv(raw_data_dir+'agg_loan_txn_acct_lvl_y3.csv', index=False)

In [ ]:
start_snap_dt='2018-01-01'
end_snap_dt='2019-07-01'
df4 = pull_agg_loan_txn(start_snap_dt, end_snap_dt)
df4.shape

In [ ]:
df4.to_csv(raw_data_dir+'agg_loan_txn_acct_lvl_y4.csv', index=False)

# Most recent Year Pull

In [13]:
start_snap_dt='2019-07-01'
end_snap_dt='2020-08-01'

In [ ]:
df5 = pull_client_general(start_snap_dt, end_snap_dt)
df5.shape

### Client General

In [ ]:
df5.drop_duplicates(subset=['mth_end_dt', 'CLNT_NO'], keep='first', inplace=True)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'clnt_general_y5.csv', index=False)

### Mort_Hist

In [ ]:
df5 = pull_mort_hist(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'mort_hist_y5.csv', index=False)

### Lend_General

In [14]:
df5 = pull_lend_general(start_snap_dt, end_snap_dt)
df5.shape

(5477538, 49)

In [15]:
df5.to_csv(raw_data_dir+'lend_general_y5.csv', index=False)

### Loan_hist

In [ ]:
df5 = pull_loan_hist(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'loan_hist_y5.csv', index=False)

### Financials

In [ ]:
df5 = pull_financials(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'financials_y5.csv', index=False)

### Fac general

In [ ]:
df5 = pull_fac_general(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'fac_general_y5.csv', index=False)

### BDA General

In [ ]:
df5 = pull_bda_general(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'bda_general_y5.csv', index=False)

### Prod Char

In [ ]:
df5 = pull_prod_char(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'prod_char_y5.csv', index=False)

### Bhvr Char

In [ ]:
df5 = pull_bhvr_char(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'bhvr_char_y5.csv', index=False)

### BRR Extract

In [ ]:
df5 = pull_brr_extract(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'brr_extract_y5.csv', index=False)

### BDA TXN NO COBC

In [ ]:
df5 = pull_bda_txn_no_cobc(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'bda_txn_no_cobc_prod_lvl_y5.csv', index=False)

### Loan General

In [ ]:
df5 = pull_loan_general(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'loan_general_y5.csv', index=False)

### Agg Loan TXN Acct Level

In [ ]:
df5 = pull_agg_loan_txn(start_snap_dt, end_snap_dt)
df5.shape

In [ ]:
df5.to_csv(raw_data_dir+'agg_loan_txn_acct_lvl_y5.csv', index=False)

In [ ]:
del df1
del df2
del df3
del df4
del df5